# Library

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime as dt
import time
import math

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [3]:
start_time = time.time()

# Fold

In [4]:
raw_fold = 'D:/data/processing/multi_road/raw'
an_fold = 'D:/data/processing/multi_road/analysis'
img_fold = 'D:/data/processing/multi_road/img'

## xlsx to csv

In [137]:
# # 4m 39.4s
# name = '매연RSD_도로측정데이터_20221004~20230910'
# file_name = f'{name}.xlsx'
# rsd = pd.read_excel(os.path.join(raw_fold, file_name))
# rsd.info()

In [138]:
# # 12.6s
# rsd.to_csv(os.path.join(raw_fold, f'{name}.csv'), index=False)

# Load

## 원격측정정보

In [5]:
rsd_cdict = {
    'vdfDateTime':'측정날짜',
    'vdfTime':'측정시간',
    'vdfRecordStatus':'기록상태',
    'vdfValid':'유효',
    'samSpeed':'속도',
    'samAccel':'가속도',
    'samFlag':'플래그',
    'vdfVSP':'VSP',
    'Smoke':'매연',
    'V_PLATE':'차량번호',
    'LineNo':'차선번호',
    'Wind speed':'풍속',
    'Humidity':'습도', 
    'Temp':'온도',
    'siteCode':'측정장소코드',
    }
len(rsd_cdict)

15

In [6]:
# 1s
name = '매연RSD_도로측정데이터_20221004~20230910'
file_name = f'{name}.csv'
rsd = pd.read_csv(os.path.join(raw_fold, file_name), usecols=rsd_cdict.keys())
rsd.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\4153831191.py:4: DtypeWarning: Columns (73) have mixed types. Specify dtype option on import or set low_memory=False.
  rsd = pd.read_csv(os.path.join(raw_fold, file_name), usecols=rsd_cdict.keys())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398673 entries, 0 to 398672
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   vdfDateTime      398673 non-null  object 
 1   vdfTime          398673 non-null  object 
 2   vdfRecordStatus  398673 non-null  object 
 3   vdfValid         398673 non-null  object 
 4   samSpeed         395272 non-null  float64
 5   samAccel         395272 non-null  float64
 6   samFlag          395272 non-null  object 
 7   vdfVSP           395272 non-null  float64
 8   Smoke            398673 non-null  float64
 9   V_PLATE          398673 non-null  object 
 10  LineNo           117903 non-null  float64
 11  Wind speed       26712 non-null   float64
 12  Humidity         99951 non-null   float64
 13  Temp             398649 non-null  float64
 14  siteCode         150 non-null     object 
dtypes: float64(8), object(7)
memory usage: 45.6+ MB


In [7]:
rsd.columns

Index(['vdfDateTime', 'vdfTime', 'vdfRecordStatus', 'vdfValid', 'samSpeed',
       'samAccel', 'samFlag', 'vdfVSP', 'Smoke', 'V_PLATE', 'LineNo',
       'Wind speed', 'Humidity', 'Temp', 'siteCode'],
      dtype='object')

In [8]:
rsdr = rsd.rename(columns=rsd_cdict)
rsdr.columns

Index(['측정날짜', '측정시간', '기록상태', '유효', '속도', '가속도', '플래그', 'VSP', '매연', '차량번호',
       '차선번호', '풍속', '습도', '온도', '측정장소코드'],
      dtype='object')

In [9]:
rsdr.head()

,측정날짜,측정시간,기록상태,유효,속도,가속도,플래그,VSP,매연,차량번호,차선번호,풍속,습도,온도,측정장소코드
0,10-04-2022,10:31:20,N,V,71.60,0.00,V,17.89,0.60,60저3406,NaN,NaN,NaN,0.00,JAS001
1,10-04-2022,10:31:46,N,V,60.70,0.00,V,2.65,0.80,41구0483,NaN,NaN,NaN,0.00,NaN
2,10-04-2022,10:31:58,N,V,50.60,0.40,V,13.42,0.70,80러0579,NaN,NaN,NaN,0.00,NaN
3,10-04-2022,10:32:47,N,V,53.30,1.40,V,18.71,0.90,94누0128,NaN,NaN,NaN,0.00,NaN
4,10-04-2022,10:34:03,N,V,36.10,0.50,V,9.51,0.70,60수4386,NaN,NaN,NaN,0.00,NaN


### [출력] csv

In [10]:
name

'매연RSD_도로측정데이터_20221004~20230910'

In [11]:
# # 1s
# rsd.to_csv(os.path.join(raw_fold, f'{name}.csv'), index=False)

## 등록정보
- 2023.07.08 기준

In [12]:
car_cdict = {
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'FRST_REG_YMD':'최초등록일자',
    'MANP_MNG_NO':'제원관리번호', 
    'MDFCN_DT':'수정일시', 
    'VHCL_ERSR_YN':'차량말소YN',
    'VHCL_FBCTN_YMD':'차량제작일자',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'VIN':'차대번호', 
    'VHRNO':'차량번호',
    # 'BSPL_STDG_CD':'법정동코드', 
    # 'PURPS_CD2':'용도', 
    # 'VHCL_MNG_NO':'차량관리번호', 
    # 'VHCL_OWNR_CL_CD':'소유자구분', 
}
len(car_cdict)

11

In [13]:
# 2m 30s
name = 'STD_CEG_CAR_MIG(20230821)'
file_name = f'{name}.csv'
car = pd.read_csv(os.path.join(raw_fold, file_name), usecols=car_cdict.keys())
car.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\2069508936.py:4: DtypeWarning: Columns (8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  car = pd.read_csv(os.path.join(raw_fold, file_name), usecols=car_cdict.keys())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30625663 entries, 0 to 30625662
Data columns (total 11 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   EXHST_GAS_CERT_NO  object
 1   EXHST_GAS_GRD_CD   object
 2   FRST_REG_YMD       object
 3   MANP_MNG_NO        object
 4   MDFCN_DT           object
 5   VHCL_ERSR_YN       object
 6   VHCL_FBCTN_YMD     object
 7   YRIDNW             int64 
 8   VHCTY_CD           object
 9   VIN                object
 10  VHRNO              object
dtypes: int64(1), object(10)
memory usage: 2.5+ GB


In [14]:
# 4s
carr = car.rename(columns=car_cdict)
carr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '제원관리번호', '수정일시', '차량말소YN', '차량제작일자',
       '차량연식', '차종', '차대번호', '차량번호'],
      dtype='object')

## 제원정보

In [15]:
src_cdict = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    # 'VHCTY_CD':'차종', 
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    # 'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력', 
}
len(src_cdict)

11

In [16]:
# 2s
name = 'STD_CEG_CAR_SRC_MIG(20230821)'
file_name = f'{name}.csv'
src = pd.read_csv(os.path.join(raw_fold, file_name), usecols=src_cdict.keys())
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566615 entries, 0 to 566614
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   VHCL_FRM      536460 non-null  object 
 1   DSPLVL        530949 non-null  float64
 2   EGIN_OTPT     491335 non-null  float64
 3   EGIN_TY       517013 non-null  object 
 4   FUEL_CD       545937 non-null  object 
 5   MNFCTR_NM     559794 non-null  object 
 6   MANP_MNG_NO   566615 non-null  object 
 7   TOTL_WGHT     563088 non-null  float64
 8   VHCNM         566544 non-null  object 
 9   VHCTY_CL_CD   551140 non-null  object 
 10  VHCTY_TY_CD2  553038 non-null  object 
dtypes: float64(3), object(8)
memory usage: 47.6+ MB


In [17]:
srcr = src.rename(columns=src_cdict)
srcr.columns

Index(['자동차형식', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '제원관리번호', '총중량', '차명',
       '차종분류', '차종유형'],
      dtype='object')

## 정밀검사

In [18]:
eet_cdict = {
    'VIN':'차대번호', 
    # 'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', # CO(매연)
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', # HC
    'PREC_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    'PREC_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', # NOx
    'PREC_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    'PREC_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    # 'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_YMD':'검사일자',
    # 'PRCINSP_DRVNG_DSTNC':'주행거리', 
}
len(eet_cdict)

12

In [20]:
# 2m 38.7s
name = 'STD_TB_EET_HIS_ME(part)(20230918)'
file_name = f'{name}.csv'
eet = pd.read_csv(os.path.join(raw_fold, file_name), usecols=eet_cdict.keys())
eet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102191938 entries, 0 to 102191937
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VIN                    object 
 1   PREC_NLOD_SMO_MSTVL1   float64
 2   PREC_NLOD_SMO_JT_YN1   object 
 3   PREC_NLOD_SMO_PRMVL1   float64
 4   PREC_NLOD_SMO_MSTVL2   float64
 5   PREC_NLOD_SMO_JT_YN2   object 
 6   PREC_NLOD_SMO_PRMVL2   float64
 7   PREC_NLOD_SMO_MSTVL3   float64
 8   PREC_NLOD_SMO_JT_YN3   object 
 9   PREC_NLOD_SMO_PRMVL3   float64
 10  PRCINSP_MSRMT_MTHD_CD  object 
 11  PRCINSP_YMD            int64  
dtypes: float64(6), int64(1), object(5)
memory usage: 9.1+ GB


In [21]:
# 12s
eetr = eet.rename(columns=eet_cdict)
eetr.columns

Index(['차대번호', '무부하매연측정치1', '무부하매연판정1', '무부하매연허용치1', '무부하매연측정치2', '무부하매연판정2',
       '무부하매연허용치2', '무부하매연측정치3', '무부하매연판정3', '무부하매연허용치3', '검사방법', '검사일자'],
      dtype='object')

In [22]:
eetr['검사일자'].describe()

count   102191938.00
mean     20150152.16
std         55240.81
min      20020520.00
25%      20101023.00
50%      20151223.00
75%      20200718.00
max      20230731.00
Name: 검사일자, dtype: float64

In [23]:
eetr.columns

Index(['차대번호', '무부하매연측정치1', '무부하매연판정1', '무부하매연허용치1', '무부하매연측정치2', '무부하매연판정2',
       '무부하매연허용치2', '무부하매연측정치3', '무부하매연판정3', '무부하매연허용치3', '검사방법', '검사일자'],
      dtype='object')

# Preprocessing

## 원격측정정보

### 측정날짜, 측정시간 전처리
- datetime type으로 변환

In [24]:
rsdr[['측정날짜', '측정시간']].head()

,측정날짜,측정시간
0,10-04-2022,10:31:20
1,10-04-2022,10:31:46
2,10-04-2022,10:31:58
3,10-04-2022,10:32:47
4,10-04-2022,10:34:03


In [25]:
rsdr[['측정날짜', '측정시간']].isnull().sum()

측정날짜    0
측정시간    0
dtype: int64

In [26]:
rsdr[['측정날짜', '측정시간']].dtypes

측정날짜    object
측정시간    object
dtype: object

#### [issue] 측정시간 값 오류
- 자동차등록번호(V_PLATE) : 서울06도5872, 대전80바9861, 경기86소7588, ********
- index : 266253, 266383, 266384, 269478

In [27]:
rsdr.loc[[266253, 266383, 266384, 269478]]

,측정날짜,측정시간,기록상태,유효,속도,가속도,플래그,VSP,매연,차량번호,차선번호,풍속,습도,온도,측정장소코드
266253,08-08-2023,0.491134259,N,V,72.36,0.00,V,18.13,1.90,서울06도5872,NaN,NaN,NaN,33.10,NaN
266383,08-08-2023,0.499652778,N,V,50.40,-1.44,V,5.57,1.70,대전80바9861,NaN,NaN,NaN,33.10,NaN
266384,08-08-2023,0.499768519,N,V,72.72,1.88,V,29.86,0.00,경기86소7588,NaN,NaN,NaN,33.10,NaN
269478,08-08-2023,0.7753125,N,V,44.28,-0.36,V,8.79,0.00,********,NaN,NaN,NaN,33.40,NaN


In [28]:
rsdr[(rsdr['차량번호'] == '서울06도5872') | (rsdr['차량번호'] == '대전80바9861') | (rsdr['차량번호'] == '경기86소7588')]

,측정날짜,측정시간,기록상태,유효,속도,가속도,플래그,VSP,매연,차량번호,차선번호,풍속,습도,온도,측정장소코드
266253,08-08-2023,0.491134259,N,V,72.36,0.00,V,18.13,1.90,서울06도5872,NaN,NaN,NaN,33.10,NaN
266383,08-08-2023,0.499652778,N,V,50.40,-1.44,V,5.57,1.70,대전80바9861,NaN,NaN,NaN,33.10,NaN
266384,08-08-2023,0.499768519,N,V,72.72,1.88,V,29.86,0.00,경기86소7588,NaN,NaN,NaN,33.10,NaN
379009,09-07-2023,11:52:47,N,V,51.48,-0.36,V,10.45,0.50,대전80바9861,3.00,0.40,53.00,27.40,NaN


In [29]:
# 41.4s
rsdr['측정날짜시간'] = rsdr['측정날짜'] + ' ' + rsdr['측정시간']
rsdr['측정날짜시간']= pd.to_datetime(rsdr['측정날짜시간'], errors='coerce')
rsdr['측정날짜시간'].head()

0   2022-10-04 10:31:20
1   2022-10-04 10:31:46
2   2022-10-04 10:31:58
3   2022-10-04 10:32:47
4   2022-10-04 10:34:03
Name: 측정날짜시간, dtype: datetime64[ns]

In [30]:
rsdr['측정날짜시간'].isnull().sum()

4

In [31]:
rsdr.loc[rsdr['측정날짜시간'].isnull() == True].index

Int64Index([266253, 266383, 266384, 269478], dtype='int64')

In [32]:
# rsdr['측정날짜시간_년'] = rsdr['측정날짜시간'].dt.year
# rsdr['측정날짜시간_월'] = rsdr['측정날짜시간'].dt.month
# rsdr['측정날짜시간_일'] = rsdr['측정날짜시간'].dt.day
# rsdr['측정날짜시간_시'] = rsdr['측정날짜시간'].dt.hour
# rsdr['측정날짜시간_분'] = rsdr['측정날짜시간'].dt.minute
# rsdr['측정날짜시간_초'] = rsdr['측정날짜시간'].dt.second
# rsdr['측정날짜시간_요일'] = rsdr['측정날짜시간'].dt.weekday.map({0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'})

In [33]:
rsdr['측정날짜'] = pd.to_datetime(rsdr['측정날짜'])

In [34]:
rsdr.head()

,측정날짜,측정시간,기록상태,유효,속도,가속도,플래그,VSP,매연,차량번호,차선번호,풍속,습도,온도,측정장소코드,측정날짜시간
0,2022-10-04,10:31:20,N,V,71.60,0.00,V,17.89,0.60,60저3406,NaN,NaN,NaN,0.00,JAS001,2022-10-04 10:31:20
1,2022-10-04,10:31:46,N,V,60.70,0.00,V,2.65,0.80,41구0483,NaN,NaN,NaN,0.00,NaN,2022-10-04 10:31:46
2,2022-10-04,10:31:58,N,V,50.60,0.40,V,13.42,0.70,80러0579,NaN,NaN,NaN,0.00,NaN,2022-10-04 10:31:58
3,2022-10-04,10:32:47,N,V,53.30,1.40,V,18.71,0.90,94누0128,NaN,NaN,NaN,0.00,NaN,2022-10-04 10:32:47
4,2022-10-04,10:34:03,N,V,36.10,0.50,V,9.51,0.70,60수4386,NaN,NaN,NaN,0.00,NaN,2022-10-04 10:34:03


In [35]:
# rsdr['측정날짜시간_요일'].unique()

In [36]:
rsdr.shape[0], len(rsdr['차량번호'].unique())

(398673, 145008)

## 등록정보

In [37]:
carr['수정일시'].dtype

dtype('O')

In [38]:
carr['수정일시'].head()

0    2021-04-28 04:04:09.000000
1    2022-10-13 04:10:15.000000
2    2020-01-14 04:01:33.000000
3                           NaN
4                           NaN
Name: 수정일시, dtype: object

In [39]:
carr['수정일시'].describe()

count                       25062854
unique                         17291
top       2023-06-14 05:06:39.000000
freq                          872014
Name: 수정일시, dtype: object

In [40]:
carr['차량말소YN'].value_counts(dropna=False)

N    25795579
Y     4830084
Name: 차량말소YN, dtype: int64

In [41]:
carr.shape

(30625663, 11)

In [42]:
# 중복 차량번호 제거(최신 수정일시 남기기)
# 2m 18s
carr = carr.sort_values('수정일시', ascending=False).drop_duplicates('차량번호').reset_index(drop=True)
carr.shape

(30460136, 11)

In [43]:
carr[carr['차량번호'].duplicated(keep=False)].shape

(0, 11)

In [44]:
carr['차량말소YN'].value_counts(dropna=False)

N    25788638
Y     4671498
Name: 차량말소YN, dtype: int64

In [45]:
# 21s
# 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['1', '3', '2', nan, '4', '5'], dtype=object)

In [46]:
# 16.1s
# 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '특수', '승합'], dtype=object)

In [47]:
carr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '제원관리번호', '수정일시', '차량말소YN', '차량제작일자',
       '차량연식', '차종', '차대번호', '차량번호'],
      dtype='object')

## 제원정보

In [48]:
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG', 
    'A90GH':'휘발유_하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유_하이브리드', 
    'A92CN':'CNG', 
    'A93HD':'수소', 
    'A92LH':'LPG_하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG_하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG', 
    'A90PH':'플러그인_하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['휘발유', '경유', '기타연료', 'LPG', nan, '휘발유_하이브리드', 'CNG', '등유', '태양열',
       '경유_하이브리드', '전기', '알코올', 'LPG_하이브리드', '수소', 'LNG', 'CNG_하이브리드'],
      dtype=object)

In [49]:
## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '중형', '소형', '경형', nan], dtype=object)

## 정밀검사

In [50]:
eetr.shape

(102191938, 12)

In [53]:
# ## 검사종류 코드 변환
# # 27.5s
# knd_dict = {
#     'A21P01':'정밀검사', 
#     'A1904':'정기검사', 
#     'A21T06':'종합검사(경과)', 
#     'A21T05':'종합검사', 
#     'A21S01':'특정검사', 
#     'A1902':'재검사(2부재검사)',
#     'A21R02':'2부재검사(종합)', 
#     'A21P02':'정밀검사(경과)', 
#     'A1903':'정기(경과)검사', 
#     'A21S04':'특정재검사(배출)', 
#     'A21E01':'배출재검사', 
#     'A21R01':'1부재검사(종합)',
#     'A21C02':'구조변경검사', 
#     'A21002':'관능재검사(임시)', 
#     'A21S03':'특정재검사(관능)', 
#     'A21S02':'특정검사(경과)', 
#     'A1901':'재검사(1부재검사)', 
#     'A21T02':'종합(정밀)',
#     'A21E02':'배출재검사(구변)', 
#     'A21T01':'종합(경과:정밀)', 
#     'A21T04':'종합(정밀) 2부재검사', 
#     'A21001':'관능재검사', 
#     'A21T03':'종합(정밀) 1부재검사', 
#     'A21TMP':'임시검사', 
# }
# eetr['검사종류'] = eetr['검사종류'].replace(knd_dict)
# eetr['검사종류'].unique()

In [52]:
## 검사방법 코드 변환
# 14.9s
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
eetr['검사방법'] = eetr['검사방법'].replace(mth_dict)
eetr['검사방법'].unique()

array(['부하검사(ASM-Idling)', '부하검사(KD-147)', '무부하검사(급가속)', '부하검사(LUG DOWN)',
       '무부하검사(TSI)', '무부하검사(정지가동)', '부하검사(ASM2525)', nan], dtype=object)

In [54]:
eetr.columns

Index(['차대번호', '무부하매연측정치1', '무부하매연판정1', '무부하매연허용치1', '무부하매연측정치2', '무부하매연판정2',
       '무부하매연허용치2', '무부하매연측정치3', '무부하매연판정3', '무부하매연허용치3', '검사방법', '검사일자'],
      dtype='object')

In [55]:
eetr['검사일자'].dtype

dtype('int64')

In [56]:
eetr['검사일자'].isnull().sum()

0

In [57]:
eetr.head()

,차대번호,무부하매연측정치1,무부하매연판정1,무부하매연허용치1,무부하매연측정치2,무부하매연판정2,무부하매연허용치2,무부하매연측정치3,무부하매연판정3,무부하매연허용치3,검사방법,검사일자
0,KNMA4C2BM5P195759,0.05,Y,0.46,3.00,Y,90.00,1.00,Y,630.00,부하검사(ASM-Idling),20230422
1,KMJHG17HPBC011031,19.00,N,15.00,NaN,NaN,NaN,NaN,NaN,NaN,부하검사(KD-147),20230422
2,KNAKU815BBA201282,0.00,Y,15.00,NaN,NaN,NaN,NaN,NaN,NaN,부하검사(KD-147),20230422
3,KNALM415BDA122710,0.02,Y,0.46,5.00,Y,37.00,1.00,Y,610.00,부하검사(ASM-Idling),20230422
4,KMHFA41CP9A441809,0.20,Y,0.46,0.00,Y,37.00,0.00,Y,610.00,부하검사(ASM-Idling),20230422


In [58]:
# 2.1s
eetr['검사일자'] = pd.to_datetime(eetr['검사일자'], format="%Y%m%d")

In [59]:
eetr['검사일자'].isnull().sum()

0

In [60]:
eetr['검사일자'].head()

0   2023-04-22
1   2023-04-22
2   2023-04-22
3   2023-04-22
4   2023-04-22
Name: 검사일자, dtype: datetime64[ns]

### 최신검사만

In [61]:
# 27.6s
# 102191938, 26723911
eetr.shape[0], len(eetr['차대번호'].unique())

(102191938, 26723911)

In [62]:
# 1m 43.4s
eetr = eetr.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
eetr.shape

(26723911, 12)

### 2022년 1월부터 사용

In [63]:
eetr['검사일자'].head()

0   2023-07-31
1   2023-07-31
2   2023-07-31
3   2023-07-31
4   2023-07-31
Name: 검사일자, dtype: datetime64[ns]

In [64]:
# 3.0s
eetr = eetr[eetr['검사일자'] >= '2022-01-01'].reset_index(drop=True)
eetr.shape

(11577336, 12)

In [65]:
eetr['차대번호'].isnull().sum()

0

## 원격&등록

In [66]:
rsdr.shape

(398673, 16)

In [67]:
rsdr.columns

Index(['측정날짜', '측정시간', '기록상태', '유효', '속도', '가속도', '플래그', 'VSP', '매연', '차량번호',
       '차선번호', '풍속', '습도', '온도', '측정장소코드', '측정날짜시간'],
      dtype='object')

In [68]:
carr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '제원관리번호', '수정일시', '차량말소YN', '차량제작일자',
       '차량연식', '차종', '차대번호', '차량번호'],
      dtype='object')

In [69]:
# 44s
rc = rsdr.merge(carr, on='차량번호', how='left')
rc.shape

(398673, 26)

In [71]:
rc[['차량번호', '차대번호']].isnull().sum()

차량번호         0
차대번호    139426
dtype: int64

In [72]:
rc.shape[0], len(rc['차대번호'].unique())

(398673, 138444)

In [76]:
rc.shape[0], len(rc['제원관리번호'].unique())

(398673, 17840)

## 원격&등록&제원

In [73]:
srcr.shape

(566615, 11)

In [74]:
srcr.columns

Index(['자동차형식', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '제원관리번호', '총중량', '차명',
       '차종분류', '차종유형'],
      dtype='object')

In [75]:
rc.shape

(398673, 26)

In [77]:
rcs = rc.merge(srcr, on='제원관리번호', how='left')
rcs.shape

(398673, 36)

In [78]:
rcs[['차량번호', '차대번호', '연료']].isnull().sum()

차량번호         0
차대번호    139426
연료      139456
dtype: int64

## 원격&등록&제원&정밀

In [79]:
rcs.shape

(398673, 36)

In [80]:
eetr.shape

(11577336, 12)

In [81]:
eetr['검사일자'].isnull().sum()

0

In [82]:
# 38.8s
rcse = rcs.merge(eetr, on='차대번호', how='left')
rcse.shape

(398673, 47)

In [83]:
rcse[['차량번호', '연료', '검사일자']].isnull().sum()

차량번호         0
연료      139456
검사일자    277297
dtype: int64

### [출력] 원격&등록&제원&정밀

In [84]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20231004'

In [85]:
rcse.columns

Index(['측정날짜', '측정시간', '기록상태', '유효', '속도', '가속도', '플래그', 'VSP', '매연', '차량번호',
       '차선번호', '풍속', '습도', '온도', '측정장소코드', '측정날짜시간', '배출가스인증번호', '배출가스등급',
       '최초등록일자', '제원관리번호', '수정일시', '차량말소YN', '차량제작일자', '차량연식', '차종', '차대번호',
       '자동차형식', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '차종분류',
       '차종유형', '무부하매연측정치1', '무부하매연판정1', '무부하매연허용치1', '무부하매연측정치2', '무부하매연판정2',
       '무부하매연허용치2', '무부하매연측정치3', '무부하매연판정3', '무부하매연허용치3', '검사방법', '검사일자'],
      dtype='object')

In [104]:
rcse_cols = [
    # 원격측정정보
    '측정날짜',
    '측정시간',
    '기록상태',
    '유효',
    '속도',
    '가속도',
    '플래그',
    'VSP',
    '매연',
    '차량번호',
    '차선번호',
    '풍속',
    '습도',
    '온도',
    '측정장소코드',
    '측정날짜시간',
    
    # 제원정보
    '배출가스인증번호', 
    '배출가스등급', 
    '최초등록일자', 
    '제원관리번호', 
    '수정일시', 
    '차량말소YN', 
    '차량제작일자', 
    '차량연식', 
    '차종', 
    '차대번호', 
    '자동차형식', 
    '배기량', 
    '엔진출력', 
    '엔진형식', 
    '연료', 
    '제작사명', 
    '총중량', 
    '차명', 
    '차종분류', 
    '차종유형', 
    
    # 정밀검사
    '무부하매연측정치1', # CO(매연)
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치2', 
    '무부하매연판정2', 
    '무부하매연허용치2', 
    '무부하매연측정치3', # NOx 
    '무부하매연판정3', 
    '무부하매연허용치3', 
    '검사방법', 
    '검사일자', 
    ]
len(rcse_cols)

47

In [107]:
# 10.6s
file_name = f'원격_등록_제원_정밀({today_date}).csv'
rcse[rcse_cols].to_csv(os.path.join(raw_fold, file_name), index=False)
print(f'{file_name} 출력 완료.')

원격_등록_제원_정밀(20231004).csv 출력 완료.


## 유효데이터만 추출
- vdfRecordStatus = N
- vdfValid = V
- samFlag = V
- 자동차등록번호 : nan, 오류 값(********) 제외
- 매연 5% 초과 제외

In [87]:
rcse['차선번호'].value_counts(dropna=False)

NaN     280770
2.00     47427
1.00     38417
3.00     32059
Name: 차선번호, dtype: int64

In [88]:
rcse.loc[rcse['차선번호'].isnull() == False, '측정날짜'].sort_values()

279391   2023-08-14
282205   2023-08-14
282206   2023-08-14
282207   2023-08-14
282208   2023-08-14
            ...    
394859   2023-09-10
394858   2023-09-10
394857   2023-09-10
394865   2023-09-10
398663   2023-09-10
Name: 측정날짜, Length: 117903, dtype: datetime64[ns]

In [89]:
temp = rcse.loc[rcse['차선번호'].isnull() == False]
temp['측정날짜'] = pd.to_datetime(temp['측정날짜'], errors='coerce')

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\3811848111.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['측정날짜'] = pd.to_datetime(temp['측정날짜'], errors='coerce')


In [90]:
temp['측정날짜'].describe()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\2014358724.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  temp['측정날짜'].describe()


count                  117903
unique                     22
top       2023-08-31 00:00:00
freq                     8673
first     2023-08-14 00:00:00
last      2023-09-10 00:00:00
Name: 측정날짜, dtype: object

In [91]:
rcse.columns

Index(['측정날짜', '측정시간', '기록상태', '유효', '속도', '가속도', '플래그', 'VSP', '매연', '차량번호',
       '차선번호', '풍속', '습도', '온도', '측정장소코드', '측정날짜시간', '배출가스인증번호', '배출가스등급',
       '최초등록일자', '제원관리번호', '수정일시', '차량말소YN', '차량제작일자', '차량연식', '차종', '차대번호',
       '자동차형식', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '차종분류',
       '차종유형', '무부하매연측정치1', '무부하매연판정1', '무부하매연허용치1', '무부하매연측정치2', '무부하매연판정2',
       '무부하매연허용치2', '무부하매연측정치3', '무부하매연판정3', '무부하매연허용치3', '검사방법', '검사일자'],
      dtype='object')

In [92]:
rcse['차량번호'].head()

0    60저3406
1    41구0483
2    80러0579
3    94누0128
4    60수4386
Name: 차량번호, dtype: object

In [93]:
rcse.shape, len(rcse['차량번호'].unique())

((398673, 47), 145008)

In [94]:
rcse['차량번호'].isnull().sum()

0

In [95]:
rcse[rcse['차량번호'] == '********'].shape

(128491, 47)

In [96]:
# N : 일반데이터, C : 교정 중 측정된자료, A : Audit데이터(표준필터 측정값)
rcse['기록상태'].value_counts(dropna=False)

N    394679
A      2617
C      1377
Name: 기록상태, dtype: int64

In [97]:
# V : 일반데이터, x : 교정중
rcse['유효'].value_counts(dropna=False)

V    397284
x      1389
Name: 유효, dtype: int64

In [98]:
# V : 일반데이터, x : 교정중
rcse['플래그'].value_counts(dropna=False)

V      367634
x       27638
NaN      3401
Name: 플래그, dtype: int64

In [99]:
rcse[['속도', '가속도', '매연']].describe()

,속도,가속도,매연
count,395272.00,395272.00,398673.00
mean,61.70,-0.49,1.22
std,25.31,3.66,6.34
min,0.00,-325.50,0.00
25%,54.36,-1.08,0.00
50%,64.44,-0.36,0.40
75%,74.52,0.00,0.90
max,2964.70,314.10,1024.20


In [100]:
rcse.loc[rcse['기록상태'] == 'N', '유효'].value_counts(dropna=False)

V    394542
x       137
Name: 유효, dtype: int64

In [101]:
rcse.loc[rcse['기록상태'] == 'N', '플래그'].value_counts(dropna=False)

V    367091
x     27588
Name: 플래그, dtype: int64

In [102]:
rcse['매연'].describe()

count   398673.00
mean         1.22
std          6.34
min          0.00
25%          0.00
50%          0.40
75%          0.90
max       1024.20
Name: 매연, dtype: float64

In [103]:
rcse.shape

(398673, 47)

In [105]:
# vdfRecordStatus = N, vdfValid = V, samFlag = V, 차량번호 : nan, 오류 값(********) 제외, 매연 5% 초과 제외
df = rcse[(rcse['기록상태'] == 'N') & (rcse['유효'] == 'V') & (rcse['플래그'] == 'V') & (rcse['차량번호'] != '********') & (rcse['차량번호'].isnull() == False) & (rcse['매연'] <= 5) & (rcse['매연'] >= 0)].reset_index(drop=True)
df.shape

(252257, 47)

In [106]:
df[['속도', '가속도', '매연']].describe()

,속도,가속도,매연
count,252257.00,252257.00,252257.00
mean,66.47,-0.51,0.64
std,14.86,1.05,0.75
min,1.00,-25.00,0.00
25%,57.24,-1.08,0.10
50%,66.24,-0.36,0.40
75%,75.96,0.00,0.90
max,119.88,25.00,5.00


### [출력] 유효데이터

In [108]:
# 8.0s
file_name = f'유효데이터(원격_등록_제원_정밀)({today_date}).csv'
df.to_csv(os.path.join(raw_fold, file_name), index=False)
print(f'{file_name} 출력 완료.')

유효데이터(원격_등록_제원_정밀)(20231004).csv 출력 완료.


# Load csv

## [로드] 원격&등록&제원&정밀

In [5]:
# # 2.4s
# name = '원격_등록_제원_정밀(20231004)'
# file_name = f'{name}.csv'
# rcse = pd.read_csv(os.path.join(raw_fold, file_name))
# rcse.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_9784\2738380534.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  rcse = pd.read_csv(os.path.join(raw_fold, file_name))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398673 entries, 0 to 398672
Data columns (total 33 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   측정날짜       398673 non-null  object 
 1   측정시간       398673 non-null  object 
 2   기록상태       398673 non-null  object 
 3   유효         398673 non-null  object 
 4   속도         395272 non-null  float64
 5   가속도        395272 non-null  float64
 6   플래그        395272 non-null  object 
 7   VSP        395272 non-null  float64
 8   매연         398673 non-null  float64
 9   자동차등록번호    398673 non-null  object 
 10  차선번호       117903 non-null  float64
 11  풍속         26712 non-null   float64
 12  습도         99951 non-null   float64
 13  온도         398649 non-null  float64
 14  측정장소코드     150 non-null     object 
 15  배출가스등급     248470 non-null  float64
 16  연료         259217 non-null  object 
 17  차량연식       259247 non-null  float64
 18  최초등록일자     259247 non-null  float64
 19  배기량        259198 non-n

## [로드] 유효데이터

In [6]:
# # 3.0s
# name = '유효데이터(원격_등록_제원_정밀)(20231004)'
# file_name = f'{name}.csv'
# df = pd.read_csv(os.path.join(raw_fold, file_name))
# df.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_9784\2626208933.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_fold, file_name))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252257 entries, 0 to 252256
Data columns (total 54 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   측정날짜       252257 non-null  object 
 1   측정시간       252257 non-null  object 
 2   기록상태       252257 non-null  object 
 3   유효         252257 non-null  object 
 4   속도         252257 non-null  float64
 5   가속도        252257 non-null  float64
 6   플래그        252257 non-null  object 
 7   VSP        252257 non-null  float64
 8   매연         252257 non-null  float64
 9   자동차등록번호    252257 non-null  object 
 10  차선번호       93271 non-null   float64
 11  풍속         21849 non-null   float64
 12  습도         80013 non-null   float64
 13  온도         252257 non-null  float64
 14  측정장소코드     53 non-null      object 
 15  측정날짜시간     252254 non-null  object 
 16  측정날짜시간_년   252254 non-null  float64
 17  측정날짜시간_월   252254 non-null  float64
 18  측정날짜시간_일   252254 non-null  float64
 19  측정날짜시간_시   252254 non-n

# Analysis

## 1\. 연료별 매연RSD 측정 평균, 건수

In [109]:
df.shape, len(df['차량번호'].unique())

((252257, 47), 138469)

In [110]:
df['연료'].isnull().sum()

9921

In [111]:
grp1 = df.groupby(['연료'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp1['매연'] = grp1['매연'].round(2)
grp1 = grp1.rename(columns={'매연':'매연RSD_평균', '차량번호':'측정건수'})
grp1.loc[grp1.shape[0]] = ['총합', np.nan, grp1['측정건수'].sum()]
grp1

,연료,매연RSD_평균,측정건수
0,CNG,1.70,939
1,LPG,0.65,19427
2,LPG_하이브리드,0.68,120
3,경유,0.65,106845
4,경유_하이브리드,0.56,553
5,기타연료,0.46,14
6,수소,0.50,539
7,전기,0.51,4649
8,휘발유,0.63,93422
9,휘발유_하이브리드,0.57,15828


## 경유차만 추출

In [112]:
df['연료'].value_counts(dropna=False).reset_index()

,index,연료
0,경유,106845
1,휘발유,93422
2,LPG,19427
3,휘발유_하이브리드,15828
4,NaN,9921
5,전기,4649
6,CNG,939
7,경유_하이브리드,553
8,수소,539
9,LPG_하이브리드,120


In [113]:
die = df[df['연료'] == '경유'].reset_index(drop=True)
die.shape

(106845, 47)

### 2\. 경유 > 차종별 매연RSD 측정 평균, 건수

In [114]:
grp2 = die.groupby(['차종'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp2['매연'] = grp2['매연'].round(2)
grp2 = grp2.rename(columns={'매연':'매연RSD_평균', '차량번호':'측정건수'})
grp2.loc[grp2.shape[0]] = ['총합', np.nan, grp2['측정건수'].sum()]
grp2

,차종,매연RSD_평균,측정건수
0,승용,0.65,57781
1,승합,0.86,6875
2,특수,0.57,1285
3,화물,0.62,40904
4,총합,NaN,106845


### 3\. 경유 > 연식별 매연RSD 측정 평균, 건수

In [115]:
grp3 = die.groupby(['연식'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp3['매연'] = grp3['매연'].round(2)
grp3 = grp3.rename(columns={'매연':'매연RSD_평균', '차량번호':'측정건수'})
grp3.loc[grp3.shape[0]] = ['총합', np.nan, grp3['측정건수'].sum()]
grp3

KeyError: '연식'

### 4\. 경유 > 차선별 매연RSD 측정 평균, 건수

In [ ]:
grp4 = die.groupby(['차선번호'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp4['매연'] = grp4['매연'].round(2)
grp4 = grp4.rename(columns={'매연':'매연RSD_평균', '차량번호':'측정건수'})
grp4.loc[grp4.shape[0]] = ['총합', np.nan, grp4['측정건수'].sum()]
grp4

,차선번호,매연RSD_평균,측정건수
0,1.00,0.80,12306
1,2.00,0.67,16293
2,3.00,0.60,10571
3,NaN,0.63,66570
4,총합,NaN,105740


### 5\. 경유 > 속도별 매연RSD 측정 평균, 건수

In [ ]:
die['속도'].describe()

count   105740.00
mean        65.89
std         14.90
min          1.00
25%         56.52
50%         65.65
75%         75.24
max        119.88
Name: 속도, dtype: float64

In [ ]:
die['속도'].isnull().sum()

0

In [116]:
math.ceil(die['속도'].max())

120

In [117]:
div = 10
n = 1
num = math.ceil(die['속도'].max())
for i in range(0, num, div):
    print(f'{n} : {i} ~ {i + div}')
    n += 1

1 : 0 ~ 10
2 : 10 ~ 20
3 : 20 ~ 30
4 : 30 ~ 40
5 : 40 ~ 50
6 : 50 ~ 60
7 : 60 ~ 70
8 : 70 ~ 80
9 : 80 ~ 90
10 : 90 ~ 100
11 : 100 ~ 110
12 : 110 ~ 120


In [118]:
# 속도 구간 나누기(10단위)
div = 10
n = 1
num = math.ceil(die['속도'].max())
for i in range(0, num, div):
    die.loc[(die['속도'] >= i) & (die['속도'] < i + div), '속도구분'] = f'{i}이상 {i + div}미만'
    die.loc[(die['속도'] >= i) & (die['속도'] < i + div), '속도구분인덱스'] = n
    n += 1

In [119]:
die['속도구분'].unique()

array(['50이상 60미만', '30이상 40미만', '40이상 50미만', '60이상 70미만', '0이상 10미만',
       '10이상 20미만', '90이상 100미만', '70이상 80미만', '20이상 30미만', '80이상 90미만',
       '100이상 110미만', '110이상 120미만'], dtype=object)

In [120]:
die['속도구분인덱스'].unique()

array([ 6.,  4.,  5.,  7.,  1.,  2., 10.,  8.,  3.,  9., 11., 12.])

In [121]:
grp5 = die.groupby(['속도구분인덱스', '속도구분'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp5['매연'] = grp5['매연'].round(2)
grp5 = grp5.rename(columns={'매연':'매연RSD_평균', '차량번호':'측정건수', '속도구분':'속도(km/h)'})
grp5.loc[grp5.shape[0]] = [grp5['속도구분인덱스'].max() + 1, '총합', np.nan, grp5['측정건수'].sum()]
grp5 = grp5.drop('속도구분인덱스', axis=1)
grp5

,속도(km/h),매연RSD_평균,측정건수
0,0이상 10미만,1.02,51
1,10이상 20미만,1.03,204
2,20이상 30미만,0.96,1213
3,30이상 40미만,0.81,3167
4,40이상 50미만,0.72,8792
5,50이상 60미만,0.66,22402
6,60이상 70미만,0.62,30465
7,70이상 80미만,0.64,23220
8,80이상 90미만,0.62,11642
9,90이상 100미만,0.61,4079


### 6\. 경유 > 배출가스등급별 매연RSD 측정 평균, 건수

In [125]:
grp6 = die.groupby(['배출가스등급'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp6['매연'] = grp6['매연'].round(2)
grp6 = grp6.rename(columns={'매연':'매연RSD_평균', '차량번호':'측정건수'})
grp6.loc[grp6.shape[0]] = ['총합', np.nan, grp6['측정건수'].sum()]
grp6

,배출가스등급,매연RSD_평균,측정건수
0,3,0.63,91575
1,4,0.83,8068
2,5,0.84,2753
3,NaN,0.58,4449
4,총합,NaN,106845


### 7\. 경유 > 배기량별 매연RSD 측정 평균, 건수

In [126]:
die['배기량'].isnull().sum()

0

In [127]:
die['배기량'].describe()

count   106845.00
mean      2684.04
std       1675.30
min       1396.00
25%       1995.00
50%       2199.00
75%       2497.00
max      16991.00
Name: 배기량, dtype: float64

In [128]:
div = 1000
n = 1
num = math.ceil(die['배기량'].max() / div)
for i in range(1, num):
    print(f'{n} : {i*div} ~ {(i + 1) * 1000}')
    n += 1

1 : 1000 ~ 2000
2 : 2000 ~ 3000
3 : 3000 ~ 4000
4 : 4000 ~ 5000
5 : 5000 ~ 6000
6 : 6000 ~ 7000
7 : 7000 ~ 8000
8 : 8000 ~ 9000
9 : 9000 ~ 10000
10 : 10000 ~ 11000
11 : 11000 ~ 12000
12 : 12000 ~ 13000
13 : 13000 ~ 14000
14 : 14000 ~ 15000
15 : 15000 ~ 16000
16 : 16000 ~ 17000


In [129]:
div = 1000
n = 1
num = math.ceil(die['배기량'].max() / div)
for i in range(1, num):
    die.loc[(die['배기량'] >= i * div) & (die['배기량'] < (i + 1) * div), '배기량구분'] = f'{i}천이상 {i + 1}천미만'
    die.loc[(die['배기량'] >= i * div) & (die['배기량'] < (i + 1) * div), '배기량구분인덱스'] = n
    n += 1

In [130]:
die['배기량구분'].unique()

array(['2천이상 3천미만', '1천이상 2천미만', '9천이상 10천미만', '12천이상 13천미만', '6천이상 7천미만',
       '3천이상 4천미만', '4천이상 5천미만', '5천이상 6천미만', '16천이상 17천미만',
       '10천이상 11천미만', '11천이상 12천미만', '8천이상 9천미만', '7천이상 8천미만',
       '15천이상 16천미만', '14천이상 15천미만'], dtype=object)

In [131]:
die['배기량구분인덱스'].unique()

array([ 2.,  1.,  9., 12.,  6.,  3.,  4.,  5., 16., 10., 11.,  8.,  7.,
       15., 14.])

In [132]:
grp7 = die.groupby(['배기량구분인덱스', '배기량구분'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp7['매연'] = grp7['매연'].round(2)
grp7 = grp7.rename(columns={'매연':'매연RSD_평균', '차량번호':'측정건수', '배기량구분':'배기량(cc)'})
grp7.loc[grp7.shape[0]] = [grp7['배기량구분인덱스'].max() + 1, '총합', np.nan, grp7['측정건수'].sum()]
grp7 = grp7.drop('배기량구분인덱스', axis=1)
grp7    

,배기량(cc),매연RSD_평균,측정건수
0,1천이상 2천미만,0.65,35240
1,2천이상 3천미만,0.65,60831
2,3천이상 4천미만,0.72,4065
3,4천이상 5천미만,0.58,227
4,5천이상 6천미만,0.50,1376
5,6천이상 7천미만,0.56,2297
6,7천이상 8천미만,0.55,126
7,8천이상 9천미만,0.72,189
8,9천이상 10천미만,1.20,745
9,10천이상 11천미만,0.34,255


### 8\. 경유 > 차량중량별 매연RSD 측정 평균, 건수

In [133]:
die['총중량'].isnull().sum()

0

In [134]:
die['총중량'].describe()

count   106845.00
mean      3583.16
std       4086.93
min       1380.00
25%       2135.00
50%       2720.00
75%       3040.00
max      39990.00
Name: 총중량, dtype: float64

In [135]:
div = 1000
n = 1
num = math.ceil(die['총중량'].max() / div)
for i in range(1, num):
    print(f'{n} : {i*div} ~ {(i + 1) * div}')
    n += 1

1 : 1000 ~ 2000
2 : 2000 ~ 3000
3 : 3000 ~ 4000
4 : 4000 ~ 5000
5 : 5000 ~ 6000
6 : 6000 ~ 7000
7 : 7000 ~ 8000
8 : 8000 ~ 9000
9 : 9000 ~ 10000
10 : 10000 ~ 11000
11 : 11000 ~ 12000
12 : 12000 ~ 13000
13 : 13000 ~ 14000
14 : 14000 ~ 15000
15 : 15000 ~ 16000
16 : 16000 ~ 17000
17 : 17000 ~ 18000
18 : 18000 ~ 19000
19 : 19000 ~ 20000
20 : 20000 ~ 21000
21 : 21000 ~ 22000
22 : 22000 ~ 23000
23 : 23000 ~ 24000
24 : 24000 ~ 25000
25 : 25000 ~ 26000
26 : 26000 ~ 27000
27 : 27000 ~ 28000
28 : 28000 ~ 29000
29 : 29000 ~ 30000
30 : 30000 ~ 31000
31 : 31000 ~ 32000
32 : 32000 ~ 33000
33 : 33000 ~ 34000
34 : 34000 ~ 35000
35 : 35000 ~ 36000
36 : 36000 ~ 37000
37 : 37000 ~ 38000
38 : 38000 ~ 39000
39 : 39000 ~ 40000


In [136]:
math.ceil(die['총중량'].max()/1000)

40

In [137]:
div = 1000
n = 1
num = math.ceil(die['총중량'].max()/div)
for i in range(0, num):
    die.loc[(die['총중량'] >= i * div) & (die['총중량'] < (i + div) * div), '총중량구분'] = f'{i}천이상 {i + 1}천미만'
    die.loc[(die['총중량'] >= i * div) & (die['총중량'] < (i + div) * div), '총중량구분인덱스'] = n
    n += 1

In [138]:
die['총중량구분'].unique()

array(['3천이상 4천미만', '2천이상 3천미만', '14천이상 15천미만', '15천이상 16천미만',
       '1천이상 2천미만', '13천이상 14천미만', '5천이상 6천미만', '10천이상 11천미만',
       '11천이상 12천미만', '6천이상 7천미만', '12천이상 13천미만', '8천이상 9천미만',
       '24천이상 25천미만', '16천이상 17천미만', '7천이상 8천미만', '9천이상 10천미만',
       '19천이상 20천미만', '39천이상 40천미만', '27천이상 28천미만', '21천이상 22천미만',
       '38천이상 39천미만', '26천이상 27천미만', '20천이상 21천미만', '18천이상 19천미만',
       '17천이상 18천미만', '25천이상 26천미만', '35천이상 36천미만', '36천이상 37천미만',
       '37천이상 38천미만', '22천이상 23천미만', '4천이상 5천미만', '31천이상 32천미만',
       '29천이상 30천미만', '33천이상 34천미만', '23천이상 24천미만', '28천이상 29천미만',
       '30천이상 31천미만', '34천이상 35천미만', '32천이상 33천미만'], dtype=object)

In [139]:
die['총중량구분인덱스'].unique()

array([ 4.,  3., 15., 16.,  2., 14.,  6., 11., 12.,  7., 13.,  9., 25.,
       17.,  8., 10., 20., 40., 28., 22., 39., 27., 21., 19., 18., 26.,
       36., 37., 38., 23.,  5., 32., 30., 34., 24., 29., 31., 35., 33.])

In [140]:
grp8 = die.groupby(['총중량구분인덱스', '총중량구분'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp8['매연'] = grp8['매연'].round(2)
grp8 = grp8.rename(columns={'총중량구분':'차량중량구분', '매연':'매연RSD_평균', '차량번호':'측정건수'})
grp8.loc[grp8.shape[0]] = [grp8['총중량구분인덱스'].max() + 1, '총합', np.nan, grp8['측정건수'].sum()]
grp8 = grp8.drop('총중량구분인덱스', axis=1)
grp8    

,차량중량구분,매연RSD_평균,측정건수
0,1천이상 2천미만,0.64,17521
1,2천이상 3천미만,0.66,57868
2,3천이상 4천미만,0.63,20616
3,4천이상 5천미만,1.12,25
4,5천이상 6천미만,0.77,1487
5,6천이상 7천미만,0.83,674
6,7천이상 8천미만,0.64,1765
7,8천이상 9천미만,0.66,402
8,9천이상 10천미만,0.91,316
9,10천이상 11천미만,0.71,837


### 9\. 경유 > 종합검사 매연허용기준별
- 매연허용치 8미만 제외
- 4륜 자동차의 경유 검사장비가 없어서 정기검사(무부하)만 가능함으로 정기검사 기준이 기록됨.(2023.09.20, 최이사님)

In [141]:
die.columns

Index(['측정날짜', '측정시간', '기록상태', '유효', '속도', '가속도', '플래그', 'VSP', '매연', '차량번호',
       '차선번호', '풍속', '습도', '온도', '측정장소코드', '측정날짜시간', '배출가스인증번호', '배출가스등급',
       '최초등록일자', '제원관리번호', '수정일시', '차량말소YN', '차량제작일자', '차량연식', '차종', '차대번호',
       '자동차형식', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '차종분류',
       '차종유형', '무부하매연측정치1', '무부하매연판정1', '무부하매연허용치1', '무부하매연측정치2', '무부하매연판정2',
       '무부하매연허용치2', '무부하매연측정치3', '무부하매연판정3', '무부하매연허용치3', '검사방법', '검사일자',
       '속도구분', '속도구분인덱스', '배기량구분', '배기량구분인덱스', '총중량구분', '총중량구분인덱스'],
      dtype='object')

In [142]:
# die['차량제작일자'].dtype

In [143]:
die['차량제작일자'] = pd.to_datetime(die['차량제작일자'], format="%Y%m%d", errors='coerce')
# die['차량제작일자'].isnull().sum()

In [144]:
# die['차량제작일자'].head()

In [145]:
# die.loc[die['차량제작일자'] == '20170904', '차량제작일자'].head()

In [146]:
# die.loc[die['차량제작일자'] == '2017-09-04', '차량제작일자'].head()

In [147]:
# die.loc[die['차량제작일자'] > '2017-09-04', '차량제작일자'].head()

In [149]:
die['검사방법'].unique()

array(['부하검사(KD-147)', '무부하검사(급가속)', nan, '부하검사(LUG DOWN)',
       '부하검사(ASM-Idling)'], dtype=object)

In [150]:
die['검사방법'].value_counts(dropna=False)

부하검사(KD-147)        47754
NaN                 45047
무부하검사(급가속)           7598
부하검사(LUG DOWN)       6423
부하검사(ASM-Idling)       23
Name: 검사방법, dtype: int64

In [151]:
die.loc[die['검사방법'] == '부하검사(KD-147)', ['검사방법', '무부하매연측정치1', '무부하매연허용치1']].head()

,검사방법,무부하매연측정치1,무부하매연허용치1
0,부하검사(KD-147),0.00,8.00
1,부하검사(KD-147),7.00,8.00
2,부하검사(KD-147),2.00,15.00
4,부하검사(KD-147),0.00,15.00
8,부하검사(KD-147),0.00,15.00


In [48]:
# die.loc[die['차량제작일자'] <= '1992-12-31', '정밀검사_매연허용치'] = 45
# die.loc[(die['차량제작일자'] >= '1993-01-01') & (die['차량제작일자'] <= '1995-12-31'), '정밀검사_매연허용치'] = 40
# die.loc[(die['차량제작일자'] >= '1996-01-01') & (die['차량제작일자'] <= '2000-12-31'), '정밀검사_매연허용치'] = 35
# die.loc[(die['차량제작일자'] >= '2001-01-01') & (die['차량제작일자'] <= '2007-12-31'), '정밀검사_매연허용치'] = 25
# die.loc[(die['차량제작일자'] >= '2008-01-01') & (die['차량제작일자'] <= '2016-08-31'), '정밀검사_매연허용치'] = 15
# die.loc[die['차량제작일자'] <= '2018-01-01', '정밀검사_매연허용치'] = 8

In [152]:
die['무부하매연허용치1'].unique()

array([ 8.  , 15.  , 20.  ,   nan, 25.  , 10.  , 40.  ,  0.49, 45.  ,
        2.18, 35.  ,  3.26,  0.56, 30.  ,  1.77, 50.  ,  2.13,  2.46])

In [153]:
die['무부하매연허용치1'].dtype

dtype('float64')

In [154]:
die['무부하매연허용치1'].describe()

count   61372.00
mean       13.88
std         4.79
min         0.49
25%         8.00
50%        15.00
75%        15.00
max        50.00
Name: 무부하매연허용치1, dtype: float64

In [155]:
die[die['무부하매연허용치1'] >= 8].shape

(61349, 53)

In [156]:
grp9 = die[die['무부하매연허용치1'] >= 8].groupby(['무부하매연허용치1'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp9['매연'] = grp9['매연'].round(2)
grp9 = grp9.rename(columns={'무부하매연허용치1':'매연허용기준(%)', '매연':'매연RSD_평균', '차량번호':'측정건수'})
grp9.loc[grp9.shape[0]] = ['총합', np.nan, grp9['측정건수'].sum()]
grp9

,매연허용기준(%),매연RSD_평균,측정건수
0,8.00,0.62,17789
1,10.00,0.61,1319
2,15.00,0.69,34155
3,20.00,0.71,3952
4,25.00,0.74,4026
5,30.00,0.69,15
6,35.00,0.40,1
7,40.00,0.98,47
8,45.00,0.82,41
9,50.00,1.90,4


### 10\. 경유 > 종합검사 NOx허용기준별

In [157]:
die['무부하매연허용치3'].unique()

array([  nan,   15.,    8.,    0.,  580.,   20., 1830.,   25., 2340.,
        700.,   30., 1240.,   50., 1200., 2070.])

In [158]:
die['무부하매연허용치3'].dtype

dtype('float64')

In [159]:
die['무부하매연허용치3'].describe()

count   28114.00
mean        4.37
std        44.17
min         0.00
25%         0.00
50%         0.00
75%         0.00
max      2340.00
Name: 무부하매연허용치3, dtype: float64

In [160]:
grp10 = die.groupby(['무부하매연허용치3'], dropna=False).agg({'매연':'mean', '차량번호':'count'}).reset_index()
grp10['매연'] = grp10['매연'].round(2)
grp10 = grp10.rename(columns={'무부하매연허용치3':'NOx허용기준(ppm)', '매연':'매연RSD_평균', '차량번호':'측정건수'})
grp10.loc[grp10.shape[0]] = ['총합', np.nan, grp10['측정건수'].sum()]
grp10

,NOx허용기준(ppm),매연RSD_평균,측정건수
0,0.00,0.66,21709
1,8.00,0.75,1393
2,15.00,0.69,4130
3,20.00,0.76,769
4,25.00,1.03,71
5,30.00,0.69,15
6,50.00,1.90,4
7,580.00,0.90,1
8,700.00,0.55,8
9,1200.00,0.03,3


## 매연RSD 매연농도별 측정건수 분포
- 농도별(10개)
    - 0 이상 0.5미만, …

In [161]:
die['매연'].describe()

count   106845.00
mean         0.65
std          0.76
min          0.00
25%          0.10
50%          0.50
75%          0.90
max          5.00
Name: 매연, dtype: float64

In [162]:
div = 0.5
n = 0
for i in range(9):
    print(f'{i + 1} : {n} ~ {n + div}')
    n += div

1 : 0 ~ 0.5
2 : 0.5 ~ 1.0
3 : 1.0 ~ 1.5
4 : 1.5 ~ 2.0
5 : 2.0 ~ 2.5
6 : 2.5 ~ 3.0
7 : 3.0 ~ 3.5
8 : 3.5 ~ 4.0
9 : 4.0 ~ 4.5


In [163]:
div = 0.5
n = 0
for i in range(9):
    die.loc[(die['매연'] >= n) & (die['매연'] < n + div), '매연구분'] = f'{n}이상 {n + div}미만'
    die.loc[(die['매연'] >= n) & (die['매연'] < n + div), '매연구분인덱스'] = i + 1
    n += div
die.loc[die['매연'] >= n, '매연구분'] = f'{n}이상'
die.loc[die['매연'] >= n, '매연구분인덱스'] = i + 2

In [164]:
die['매연구분'].unique()

array(['0.5이상 1.0미만', '1.0이상 1.5미만', '1.5이상 2.0미만', '0이상 0.5미만',
       '2.5이상 3.0미만', '3.5이상 4.0미만', '2.0이상 2.5미만', '3.0이상 3.5미만',
       '4.0이상 4.5미만', '4.5이상'], dtype=object)

In [165]:
die['매연구분인덱스'].unique()

array([ 2.,  3.,  4.,  1.,  6.,  8.,  5.,  7.,  9., 10.])

### 11\. RSD 매연 농도별 데이터 수량

In [166]:
grp11 = die.groupby(['매연구분인덱스', '매연구분'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp11 = grp11.rename(columns={'차량번호':'측정건수', '매연구분':'매연(%)'})
grp11.loc[grp11.shape[0]] = [grp11['매연구분인덱스'].max() + 1, '총합', grp11['측정건수'].sum()]
grp11 = grp11.drop('매연구분인덱스', axis=1)
grp11

,매연(%),측정건수
0,0이상 0.5미만,52595
1,0.5이상 1.0미만,30398
2,1.0이상 1.5미만,12682
3,1.5이상 2.0미만,4832
4,2.0이상 2.5미만,2370
5,2.5이상 3.0미만,1342
6,3.0이상 3.5미만,917
7,3.5이상 4.0미만,679
8,4.0이상 4.5미만,540
9,4.5이상,490


### 12\. RSD 매연 농도, 차종별 데이터 수량

In [167]:
grp12 = die.groupby(['매연구분인덱스', '매연구분', '차종'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp12 = grp12.rename(columns={'차량번호':'측정건수(건)', '매연구분':'매연(%)'})
grp12.loc[grp12.shape[0]] = [grp12['매연구분인덱스'].max() + 1, '총합', np.nan, grp12['측정건수(건)'].sum()]
grp12 = grp12.drop('매연구분인덱스', axis=1)
grp12

,매연(%),차종,측정건수(건)
0,0이상 0.5미만,승용,28310
1,0이상 0.5미만,승합,2658
2,0이상 0.5미만,특수,725
3,0이상 0.5미만,화물,20902
4,0.5이상 1.0미만,승용,16561
5,0.5이상 1.0미만,승합,2026
6,0.5이상 1.0미만,특수,312
7,0.5이상 1.0미만,화물,11499
8,1.0이상 1.5미만,승용,6899
9,1.0이상 1.5미만,승합,982


In [168]:
grp12 = die.groupby(['매연구분인덱스', '매연구분', '차종'], dropna=False).agg({'차량번호':'count'}).unstack('차종').reset_index()
grp12 = grp12.rename(columns={'매연구분':'매연(%)', '차량번호':'차종'})
grp12['합계'] = grp12.iloc[:, 2:].sum(axis=1)
grp12.loc[grp12.shape[0]] = [grp12['매연구분인덱스'].max() + 1, '총합'] + grp12.iloc[:, 2:].sum().to_list()
grp12 = grp12.drop('매연구분인덱스', axis=1, level=0)
grp12

매연(%)     차종                         합계
차종                  승용    승합    특수     화물        
0     0이상 0.5미만  28310  2658   725  20902   52595
1   0.5이상 1.0미만  16561  2026   312  11499   30398
2   1.0이상 1.5미만   6899   982   133   4668   12682
3   1.5이상 2.0미만   2680   484    41   1627    4832
4   2.0이상 2.5미만   1287   240    26    817    2370
5   2.5이상 3.0미만    704   168    10    460    1342
6   3.0이상 3.5미만    466   103    15    333     917
7   3.5이상 4.0미만    359    83     9    228     679
8   4.0이상 4.5미만    270    69     5    196     540
9         4.5이상    245    62     9    174     490
10           총합  57781  6875  1285  40904  106845

### 13\. RSD 매연 농도, 연식별 데이터 수량

In [170]:
die['차량연식'].describe()

count   106845.00
mean      2016.56
std          4.18
min       1983.00
25%       2014.00
50%       2017.00
75%       2020.00
max       2024.00
Name: 차량연식, dtype: float64

In [171]:
die.loc[die['차량연식'] == 2024, ['연료', '차명', '차량연식', '차대번호', '차량말소YN']]

,연료,차명,차량연식,차대번호,차량말소YN
8015,경유,마이티,2024.00,KMFHA17GPRC076943,N
13447,경유,마이티,2024.00,KMFHA17GPRC078773,N
16037,경유,더쎈,2024.00,KLTEE5AH2RK000687,N
16157,경유,마이티,2024.00,KMFHA17GPRC077101,N
16159,경유,더쎈,2024.00,KLTEE5AH2RK000698,N
...,...,...,...,...,...
104341,경유,렉스턴스포츠 쿨멘,2024.00,KPADA4AE1RP215951,N
105232,경유,렉스턴스포츠 쿨멘 칸,2024.00,KPADM4AE1RP216099,N
106179,경유,더쎈윙바디,2024.00,KLTEE5AHJRK000029,N
106278,경유,유니버스(UNIVERSE),2024.00,KMJKG18KPRC002556,N


In [172]:
die.loc[die['차량연식'] == 2024, '차량말소YN'].value_counts(dropna=False)

N    137
Name: 차량말소YN, dtype: int64

In [173]:
grp13 = die.groupby(['매연구분인덱스', '매연구분', '차량연식'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp13 = grp13.rename(columns={'차량번호':'측정건수(건)', '매연구분':'매연(%)'})
grp13.loc[grp13.shape[0]] = [grp13['매연구분인덱스'].max() + 1, '총합', np.nan, grp13['측정건수(건)'].sum()]
grp13 = grp13.drop('매연구분인덱스', axis=1)
grp13

,매연(%),차량연식,측정건수(건)
0,0이상 0.5미만,1983.00,1
1,0이상 0.5미만,1990.00,2
2,0이상 0.5미만,1991.00,1
3,0이상 0.5미만,1992.00,1
4,0이상 0.5미만,1994.00,1
...,...,...,...
275,4.5이상,2020.00,33
276,4.5이상,2021.00,27
277,4.5이상,2022.00,24
278,4.5이상,2023.00,20


In [174]:
grp13 = die.groupby(['매연구분인덱스', '매연구분', '차량연식'], dropna=False).agg({'차량번호':'count'}).unstack('차량연식').reset_index()
grp13 = grp13.rename(columns={'매연구분':'매연(%)', '차량번호':'차량연식(년)'})
grp13 = grp13.rename_axis(columns={'차량연식':'차량연식(년)'})
grp13['합계'] = grp13.iloc[:, 2:].sum(axis=1)
grp13.loc[grp13.shape[0]] = [grp13['매연구분인덱스'].max() + 1, '총합'] + grp13.iloc[:, 2:].sum().to_list()
grp13 = grp13.drop('매연구분인덱스', axis=1, level=0)
grp13

매연(%) 차량연식(년)                                                   \
차량연식(년)               1983.0 1990.0 1991.0 1992.0 1993.0 1994.0 1995.0 1996.0   
0          0이상 0.5미만    1.00   2.00   1.00   1.00    NaN   1.00  13.00  10.00   
1        0.5이상 1.0미만     NaN    NaN    NaN    NaN    NaN    NaN   1.00    NaN   
2        1.0이상 1.5미만     NaN    NaN    NaN    NaN    NaN    NaN   1.00    NaN   
3        1.5이상 2.0미만     NaN    NaN    NaN    NaN    NaN    NaN    NaN   3.00   
4        2.0이상 2.5미만     NaN    NaN    NaN    NaN    NaN    NaN    NaN   1.00   
5        2.5이상 3.0미만     NaN    NaN    NaN    NaN    NaN    NaN    NaN   2.00   
6        3.0이상 3.5미만     NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
7        3.5이상 4.0미만     NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
8        4.0이상 4.5미만     NaN    NaN    NaN    NaN   1.00    NaN   3.00   1.00   
9              4.5이상     NaN    NaN   1.00    NaN    NaN    NaN    NaN    NaN   
10                총합    1.00   2.00   2.00   1.00   1.00   1.00  18.00  17.00   

                ...                                                     \
차량연식(년) 1997.0  ...   2016.0  2017.0   2018.0   2019.0  2020.0  2021.0   
0        16.00  ...  5192.00 4279.00  5423.00  5852.00 4488.00 3619.00   
1         4.00  ...  3147.00 2590.00  3092.00  3327.00 2345.00 1887.00   
2         3.00  ...  1234.00 1073.00  1246.00  1251.00  855.00  761.00   
3         4.00  ...   449.00  395.00   460.00   476.00  328.00  300.00   
4         1.00  ...   208.00  211.00   213.00   212.00  137.00  145.00   
5         1.00  ...   115.00  111.00   128.00   109.00   79.00   76.00   
6          NaN  ...    75.00   72.00    83.00    85.00   58.00   54.00   
7         1.00  ...    60.00   51.00    56.00    62.00   36.00   50.00   
8          NaN  ...    44.00   34.00    38.00    34.00   30.00   33.00   
9         1.00  ...    45.00   38.00    39.00    44.00   33.00   27.00   
10       31.00  ... 10569.00 8854.00 10778.00 11452.00 8389.00 6952.00   

                                      합계  
차량연식(년)  2022.0  2023.0 2024.0            
0       3769.00 2568.00  82.00  52595.00  
1       1932.00 1294.00  28.00  30398.00  
2        751.00  494.00  13.00  12682.00  
3        247.00  160.00   9.00   4832.00  
4        141.00   74.00   2.00   2370.00  
5         71.00   48.00    NaN   1342.00  
6         49.00   41.00    NaN    917.00  
7         27.00   32.00   1.00    679.00  
8         35.00   23.00   2.00    540.00  
9         24.00   20.00    NaN    490.00  
10      7046.00 4754.00 137.00 106845.00  

[11 rows x 38 columns]

### 14\. RSD 매연 농도, 측정속도별 데이터 수량

In [175]:
die['속도구분'].unique()

array(['50이상 60미만', '30이상 40미만', '40이상 50미만', '60이상 70미만', '0이상 10미만',
       '10이상 20미만', '90이상 100미만', '70이상 80미만', '20이상 30미만', '80이상 90미만',
       '100이상 110미만', '110이상 120미만'], dtype=object)

In [176]:
die['속도구분인덱스'].unique()

array([ 6.,  4.,  5.,  7.,  1.,  2., 10.,  8.,  3.,  9., 11., 12.])

In [177]:
grp14 = die.groupby(['매연구분인덱스', '매연구분', '속도구분인덱스', '속도구분'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp14 = grp14.rename(columns={'차량번호':'측정건수(건)', '매연구분':'매연(%)', '속도구분':'속도(km/h)'})
grp14.loc[grp14.shape[0]] = [grp14['매연구분인덱스'].max() + 1, '총합', np.nan, np.nan, grp14['측정건수(건)'].sum()]
grp14 = grp14.drop(['매연구분인덱스', '속도구분인덱스'], axis=1)
grp14

,매연(%),속도(km/h),측정건수(건)
0,0이상 0.5미만,0이상 10미만,19
1,0이상 0.5미만,10이상 20미만,77
2,0이상 0.5미만,20이상 30미만,462
3,0이상 0.5미만,30이상 40미만,1310
4,0이상 0.5미만,40이상 50미만,4033
...,...,...,...
112,4.5이상,80이상 90미만,44
113,4.5이상,90이상 100미만,9
114,4.5이상,100이상 110미만,7
115,4.5이상,110이상 120미만,1


In [178]:
grp14 = die.groupby(['매연구분인덱스', '매연구분', '속도구분인덱스', '속도구분'], dropna=False).agg({'차량번호':'count'}).unstack(['속도구분인덱스', '속도구분']).reset_index()
grp14 = grp14.rename(columns={'매연구분':'매연(%)', '차량번호':'속도(km/h)'})
grp14 = grp14.rename_axis(columns={'속도구분':'속도(km/h)'})
grp14['합계'] = grp14.iloc[:, 2:].sum(axis=1)
grp14.loc[grp14.shape[0]] = [grp14['매연구분인덱스'].max() + 1, '총합'] + grp14.iloc[:, 2:].sum().to_list()
grp14 = grp14.drop('매연구분인덱스', axis=1, level=0)
grp14.columns = grp14.columns.droplevel(1)
grp14

매연(%) 속도(km/h)                                          \
속도(km/h)              0이상 10미만 10이상 20미만 20이상 30미만 30이상 40미만 40이상 50미만   
0           0이상 0.5미만    19.00     77.00    462.00   1310.00   4033.00   
1         0.5이상 1.0미만    17.00     43.00    300.00    897.00   2719.00   
2         1.0이상 1.5미만     4.00     33.00    171.00    444.00    964.00   
3         1.5이상 2.0미만     4.00     22.00     99.00    222.00    422.00   
4         2.0이상 2.5미만      NaN      6.00     70.00    110.00    211.00   
5         2.5이상 3.0미만     2.00      4.00     37.00     72.00    126.00   
6         3.0이상 3.5미만      NaN      9.00     32.00     31.00    127.00   
7         3.5이상 4.0미만     2.00      3.00     17.00     32.00     81.00   
8         4.0이상 4.5미만     1.00      3.00     11.00     28.00     59.00   
9               4.5이상     2.00      4.00     14.00     21.00     50.00   
10                 총합    51.00    204.00   1213.00   3167.00   8792.00   

                                                                         \
속도(km/h) 50이상 60미만 60이상 70미만 70이상 80미만 80이상 90미만 90이상 100미만 100이상 110미만   
0         10987.00  15465.00  11650.00   5819.00    2001.00      648.00   
1          6476.00   8425.00   6493.00   3362.00    1227.00      340.00   
2          2564.00   3655.00   2713.00   1397.00     494.00      193.00   
3           962.00   1303.00   1053.00    482.00     176.00       71.00   
4           510.00    614.00    488.00    245.00      90.00       24.00   
5           282.00    345.00    291.00    124.00      39.00       15.00   
6           220.00    236.00    169.00     69.00      18.00        5.00   
7           164.00    168.00    140.00     55.00      13.00        4.00   
8           115.00    139.00    122.00     45.00      12.00        5.00   
9           122.00    115.00    101.00     44.00       9.00        7.00   
10        22402.00  30465.00  23220.00  11642.00    4079.00     1312.00   

                            합계  
속도(km/h) 110이상 120미만            
0             124.00  52595.00  
1              99.00  30398.00  
2              50.00  12682.00  
3              16.00   4832.00  
4               2.00   2370.00  
5               5.00   1342.00  
6               1.00    917.00  
7                NaN    679.00  
8                NaN    540.00  
9               1.00    490.00  
10            298.00 106845.00

### 15\. RSD 매연 농도, 배출가스 등급별 데이터 수량

In [179]:
grp15 = die.groupby(['매연구분인덱스', '매연구분', '배출가스등급'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp15 = grp15.rename(columns={'차량번호':'측정건수(건)', '매연구분':'매연(%)'})
grp15.loc[grp15.shape[0]] = [grp15['매연구분인덱스'].max() + 1, '총합', np.nan, grp15['측정건수(건)'].sum()]
grp15 = grp15.drop('매연구분인덱스', axis=1)
grp15

,매연(%),배출가스등급,측정건수(건)
0,0이상 0.5미만,3,45879
1,0이상 0.5미만,4,3089
2,0이상 0.5미만,5,1204
3,0이상 0.5미만,NaN,2423
4,0.5이상 1.0미만,3,26130
5,0.5이상 1.0미만,4,2429
6,0.5이상 1.0미만,5,645
7,0.5이상 1.0미만,NaN,1194
8,1.0이상 1.5미만,3,10565
9,1.0이상 1.5미만,4,1250


In [181]:
grp15 = die.groupby(['매연구분인덱스', '매연구분', '배출가스등급'], dropna=False).agg({'차량번호':'count'}).unstack('배출가스등급').reset_index()
grp15 = grp15.rename(columns={'매연구분':'매연(%)', '차량번호':'배출가스등급'})
grp15['합계'] = grp15.iloc[:, 2:].sum(axis=1)
grp15.loc[grp15.shape[0]] = [grp15['매연구분인덱스'].max() + 1, '총합'] + grp15.iloc[:, 2:].sum().to_list()
grp15 = grp15.drop('매연구분인덱스', axis=1)
grp15

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\1300783737.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grp15 = grp15.drop('매연구분인덱스', axis=1)


매연(%) 배출가스등급                        합계
배출가스등급                   3     4     5   NaN        
0         0이상 0.5미만  45879  3089  1204  2423   52595
1       0.5이상 1.0미만  26130  2429   645  1194   30398
2       1.0이상 1.5미만  10565  1250   407   460   12682
3       1.5이상 2.0미만   3943   547   188   154    4832
4       2.0이상 2.5미만   1894   290   108    78    2370
5       2.5이상 3.0미만   1092   145    64    41    1342
6       3.0이상 3.5미만    730   107    43    37     917
7       3.5이상 4.0미만    536    85    35    23     679
8       4.0이상 4.5미만    411    71    36    22     540
9             4.5이상    395    55    23    17     490
10               총합  91575  8068  2753  4449  106845

### 16\. RSD 매연 농도, 배기량별 데이터 수량

In [182]:
grp16 = die.groupby(['매연구분인덱스', '매연구분', '배기량구분인덱스', '배기량구분'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp16 = grp16.rename(columns={'차량번호':'측정건수(건)', '매연구분':'매연(%)', '배기량구분':'배기량(cc)'})
grp16.loc[grp16.shape[0]] = [grp16['매연구분인덱스'].max() + 1, '총합', np.nan, np.nan, grp16['측정건수(건)'].sum()]
grp16 = grp16.drop(['매연구분인덱스', '배기량구분인덱스'], axis=1)
grp16

,매연(%),배기량(cc),측정건수(건)
0,0이상 0.5미만,1천이상 2천미만,17220
1,0이상 0.5미만,2천이상 3천미만,29719
2,0이상 0.5미만,3천이상 4천미만,1852
3,0이상 0.5미만,4천이상 5천미만,124
4,0이상 0.5미만,5천이상 6천미만,855
...,...,...,...
122,4.5이상,9천이상 10천미만,20
123,4.5이상,10천이상 11천미만,3
124,4.5이상,12천이상 13천미만,7
125,4.5이상,14천이상 15천미만,1


In [183]:
grp16 = die.groupby(['매연구분인덱스', '매연구분', '배기량구분인덱스', '배기량구분'], dropna=False).agg({'차량번호':'count'}).unstack(['배기량구분인덱스', '배기량구분']).reset_index()
grp16 = grp16.rename(columns={'매연구분':'매연(%)', '배기량구분':'배기량(cc)', '차량번호':'배기량(cc)'})
grp16 = grp16.rename_axis(columns={'배기량구분':'배기량(cc)'})
# grp16 = grp16.rename(columns={'차량번호':'배기량(cc)'}, level=0)
grp16['합계'] = grp16.iloc[:, 2:].sum(axis=1)
grp16.loc[grp16.shape[0]] = [grp16['매연구분인덱스'].max() + 1, '총합'] + grp16.iloc[:, 2:].sum().to_list()
grp16 = grp16.drop('매연구분인덱스', axis=1)
grp16.columns = grp16.columns.droplevel(1)
grp16 = grp16.T
grp16

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\2848121395.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grp16 = grp16.drop('매연구분인덱스', axis=1)


0            1            2            3   \
        배기량(cc)                                                         
매연(%)                0이상 0.5미만  0.5이상 1.0미만  1.0이상 1.5미만  1.5이상 2.0미만   
배기량(cc) 1천이상 2천미만     17220.00     10160.00      4240.00      1613.00   
        2천이상 3천미만     29719.00     17838.00      7187.00      2658.00   
        3천이상 4천미만      1852.00      1155.00       536.00       220.00   
        4천이상 5천미만       124.00        56.00        24.00        13.00   
        5천이상 6천미만       855.00       267.00       134.00        39.00   
        6천이상 7천미만      1361.00       454.00       228.00       102.00   
        7천이상 8천미만        85.00        18.00         5.00         4.00   
        8천이상 9천미만       103.00        27.00        24.00        18.00   
        9천이상 10천미만      218.00       172.00       134.00        70.00   
        10천이상 11천미만     202.00        19.00        11.00         9.00   
        11천이상 12천미만     123.00        11.00        10.00        10.00   
        12천이상 13천미만     686.00       220.00       148.00        74.00   
        14천이상 15천미만      10.00          NaN          NaN         2.00   
        15천이상 16천미만      14.00         1.00          NaN          NaN   
        16천이상 17천미만      23.00          NaN         1.00          NaN   
합계                    52595.00     30398.00     12682.00      4832.00   

                              4            5            6            7   \
        배기량(cc)                                                           
매연(%)                2.0이상 2.5미만  2.5이상 3.0미만  3.0이상 3.5미만  3.5이상 4.0미만   
배기량(cc) 1천이상 2천미만         793.00       450.00       271.00       196.00   
        2천이상 3천미만        1263.00       707.00       504.00       385.00   
        3천이상 4천미만         113.00        55.00        46.00        31.00   
        4천이상 5천미만           5.00         1.00         2.00          NaN   
        5천이상 6천미만          24.00        23.00        17.00         7.00   
        6천이상 7천미만          54.00        30.00        24.00        15.00   
        7천이상 8천미만           6.00         2.00         2.00         1.00   
        8천이상 9천미만           5.00         4.00         3.00         1.00   
        9천이상 10천미만         48.00        30.00        21.00        22.00   
        10천이상 11천미만         5.00         3.00         1.00         1.00   
        11천이상 12천미만         6.00         3.00         2.00         1.00   
        12천이상 13천미만        48.00        34.00        24.00        19.00   
        14천이상 15천미만          NaN          NaN          NaN          NaN   
        15천이상 16천미만          NaN          NaN          NaN          NaN   
        16천이상 17천미만          NaN          NaN          NaN          NaN   
합계                       2370.00      1342.00       917.00       679.00   

                              8      9         10  
        배기량(cc)                                    
매연(%)                4.0이상 4.5미만  4.5이상        총합  
배기량(cc) 1천이상 2천미만         156.00 141.00  35240.00  
        2천이상 3천미만         303.00 267.00  60831.00  
        3천이상 4천미만          27.00  30.00   4065.00  
        4천이상 5천미만           1.00   1.00    227.00  
        5천이상 6천미만           5.00   5.00   1376.00  
        6천이상 7천미만          18.00  11.00   2297.00  
        7천이상 8천미만           2.00   1.00    126.00  
        8천이상 9천미만           1.00   3.00    189.00  
        9천이상 10천미만         10.00  20.00    745.00  
        10천이상 11천미만         1.00   3.00    255.00  
        11천이상 12천미만         1.00    NaN    167.00  
        12천이상 13천미만        13.00   7.00   1273.00  
        14천이상 15천미만          NaN   1.00     13.00  
        15천이상 16천미만          NaN    NaN     15.00  
        16천이상 17천미만         2.00    NaN     26.00  
합계                        540.00 490.00 106845.00

### 17\. RSD 매연 농도, 차량중량별 데이터 수량

In [184]:
grp17 = die.groupby(['매연구분인덱스', '매연구분', '총중량구분인덱스', '총중량구분'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp17 = grp17.rename(columns={'총중량구분':'차량중량(kg)', '차량번호':'측정건수(건)', '매연구분':'매연(%)'})
grp17.loc[grp17.shape[0]] = [grp17['매연구분인덱스'].max() + 1, '총합', np.nan, np.nan, grp17['측정건수(건)'].sum()]
grp17 = grp17.drop(['매연구분인덱스', '총중량구분인덱스'], axis=1)
grp17

,매연(%),차량중량(kg),측정건수(건)
0,0이상 0.5미만,1천이상 2천미만,8614
1,0이상 0.5미만,2천이상 3천미만,28100
2,0이상 0.5미만,3천이상 4천미만,10188
3,0이상 0.5미만,4천이상 5천미만,7
4,0이상 0.5미만,5천이상 6천미만,601
...,...,...,...
250,4.5이상,14천이상 15천미만,9
251,4.5이상,15천이상 16천미만,7
252,4.5이상,16천이상 17천미만,3
253,4.5이상,37천이상 38천미만,1


In [185]:
grp17 = die.groupby(['매연구분인덱스', '매연구분', '총중량구분인덱스', '총중량구분'], dropna=False).agg({'차량번호':'count'}).unstack(['총중량구분인덱스', '총중량구분']).reset_index()
grp17 = grp17.rename(columns={'매연구분':'매연(%)', '차량번호':'차량중량(kg)'})
grp17 = grp17.rename_axis(columns={'총중량구분':'차량중량(kg)'})
grp17['합계'] = grp17.iloc[:, 2:].sum(axis=1)
grp17.loc[grp17.shape[0]] = [grp17['매연구분인덱스'].max() + 1, '총합'] + grp17.iloc[:, 2:].sum().to_list()
grp17 = grp17.drop('매연구분인덱스', axis=1)
grp17.columns = grp17.columns.droplevel(1)
grp17 = grp17.T
grp17

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\2008584361.py:6: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grp17 = grp17.drop('매연구분인덱스', axis=1)


0            1            2            3   \
         차량중량(kg)                                                        
매연(%)                 0이상 0.5미만  0.5이상 1.0미만  1.0이상 1.5미만  1.5이상 2.0미만   
차량중량(kg) 1천이상 2천미만      8614.00      5095.00      2075.00       830.00   
         2천이상 3천미만     28100.00     16714.00      6969.00      2610.00   
         3천이상 4천미만     10188.00      6171.00      2378.00       827.00   
         4천이상 5천미만         7.00         6.00         5.00         4.00   
         5천이상 6천미만       601.00       464.00       215.00        94.00   
         6천이상 7천미만       287.00       178.00       104.00        43.00   
         7천이상 8천미만       905.00       476.00       193.00        80.00   
         8천이상 9천미만       203.00       110.00        47.00        16.00   
         9천이상 10천미만      147.00        62.00        36.00        23.00   
         10천이상 11천미만     408.00       198.00       115.00        42.00   
         11천이상 12천미만     453.00       142.00        70.00        25.00   
         12천이상 13천미만     368.00        73.00        42.00         7.00   
         13천이상 14천미만     365.00       145.00        90.00        55.00   
         14천이상 15천미만     363.00       232.00       136.00        66.00   
         15천이상 16천미만     290.00       188.00       139.00        70.00   
         16천이상 17천미만     170.00        23.00         7.00        10.00   
         17천이상 18천미만      46.00         9.00         2.00         4.00   
         18천이상 19천미만      32.00         4.00         3.00         3.00   
         19천이상 20천미만      25.00         4.00          NaN          NaN   
         20천이상 21천미만      11.00         2.00         1.00          NaN   
         21천이상 22천미만      16.00         5.00          NaN         3.00   
         22천이상 23천미만      34.00         4.00         1.00         1.00   
         23천이상 24천미만      13.00          NaN          NaN         1.00   
         24천이상 25천미만      29.00         1.00         1.00          NaN   
         25천이상 26천미만     206.00        16.00        13.00         2.00   
         26천이상 27천미만      79.00         7.00         6.00         1.00   
         27천이상 28천미만      18.00         5.00         1.00          NaN   
         28천이상 29천미만       7.00          NaN          NaN          NaN   
         29천이상 30천미만      13.00          NaN         2.00          NaN   
         30천이상 31천미만       9.00         1.00         2.00         1.00   
         31천이상 32천미만      12.00          NaN         1.00          NaN   
         32천이상 33천미만       4.00          NaN          NaN          NaN   
         33천이상 34천미만      10.00          NaN          NaN          NaN   
         34천이상 35천미만       8.00         2.00          NaN         1.00   
         35천이상 36천미만      63.00         8.00         7.00         3.00   
         36천이상 37천미만      19.00         2.00         1.00         1.00   
         37천이상 38천미만      53.00         5.00         4.00         3.00   
         38천이상 39천미만     188.00        16.00         5.00         2.00   
         39천이상 40천미만     231.00        30.00        11.00         4.00   
합계                     52595.00     30398.00     12682.00      4832.00   

                               4            5            6            7   \
         차량중량(kg)                                                          
매연(%)                 2.0이상 2.5미만  2.5이상 3.0미만  3.0이상 3.5미만  3.5이상 4.0미만   
차량중량(kg) 1천이상 2천미만         370.00       198.00       128.00        88.00   
         2천이상 3천미만        1303.00       727.00       481.00       396.00   
         3천이상 4천미만         383.00       232.00       166.00        97.00   
         4천이상 5천미만           1.00          NaN          NaN          NaN   
         5천이상 6천미만          49.00        24.00        18.00         9.00   
         6천이상 7천미만          21.00         9.00         9.00         7.00   
         7천이상 8천미만          39.00        17.00        19.00        15.00   
         8천이상 9천미만           8.00         6.00         2.00         1.00   
         9

### 18\. RSD 매연 농도, 종합검사 매연허용기준별 데이터 수량
- 매연허용치 8미만 제외
- 4륜 자동차의 경유 검사장비가 없어서 정기검사(무부하)만 가능함으로 정기검사 기준이 기록됨.(2023.09.20, 최이사님)

In [186]:
die[die['무부하매연허용치1'] >= 8].shape

(61349, 55)

In [187]:
grp18 = die[die['무부하매연허용치1'] >= 8].groupby(['매연구분인덱스', '매연구분', '무부하매연허용치1'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp18 = grp18.rename(columns={'무부하매연허용치1':'매연허용기준(%)', '차량번호':'측정건수(건)', '매연구분':'매연(%)'})
grp18.loc[grp18.shape[0]] = [grp18['매연구분인덱스'].max() + 1, '총계', np.nan, grp18['측정건수(건)'].sum()]
grp18 = grp18.drop('매연구분인덱스', axis=1)
grp18

,매연(%),매연허용기준(%),측정건수(건)
0,0이상 0.5미만,8.00,8809
1,0이상 0.5미만,10.00,692
2,0이상 0.5미만,15.00,16005
3,0이상 0.5미만,20.00,1905
4,0이상 0.5미만,25.00,1830
...,...,...,...
68,4.5이상,15.00,180
69,4.5이상,20.00,24
70,4.5이상,25.00,31
71,4.5이상,50.00,1


In [188]:
grp18 = die[die['무부하매연허용치1'] >= 8].groupby(['매연구분인덱스', '매연구분', '무부하매연허용치1'], dropna=False).agg({'차량번호':'count'}).unstack('무부하매연허용치1').reset_index()
grp18 = grp18.rename(columns={'매연구분':'매연(%)', '차량번호':'매연허용기준(%)'})
grp18 = grp18.rename_axis(columns={'무부하매연허용치1':'매연허용기준(%)'})
grp18['합계'] = grp18.iloc[:, 2:].sum(axis=1)
grp18.loc[grp18.shape[0]] = [grp18['매연구분인덱스'].max() + 1, '총합'] + grp18.iloc[:, 2:].sum().to_list()
grp18 = grp18.drop('매연구분인덱스', axis=1)
grp18

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\1981776434.py:6: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grp18 = grp18.drop('매연구분인덱스', axis=1)


매연(%) 매연허용기준(%)                                              \
매연허용기준(%)                    8.0    10.0     15.0    20.0    25.0  30.0 35.0   
0            0이상 0.5미만   8809.00  692.00 16005.00 1905.00 1830.00  7.00 1.00   
1          0.5이상 1.0미만   5263.00  358.00  9898.00 1010.00 1123.00  2.00  NaN   
2          1.0이상 1.5미만   2061.00  146.00  4331.00  519.00  538.00  3.00  NaN   
3          1.5이상 2.0미만    733.00   53.00  1658.00  218.00  228.00  3.00  NaN   
4          2.0이상 2.5미만    350.00   31.00   860.00  112.00  104.00   NaN  NaN   
5          2.5이상 3.0미만    213.00   10.00   468.00   56.00   57.00   NaN  NaN   
6          3.0이상 3.5미만    139.00   11.00   316.00   44.00   52.00   NaN  NaN   
7          3.5이상 4.0미만     93.00    5.00   234.00   33.00   29.00   NaN  NaN   
8          4.0이상 4.5미만     67.00    4.00   205.00   31.00   34.00   NaN  NaN   
9                4.5이상     61.00    9.00   180.00   24.00   31.00   NaN  NaN   
10                  총합  17789.00 1319.00 34155.00 3952.00 4026.00 15.00 1.00   

                                 합계  
매연허용기준(%)  40.0  45.0 50.0           
0          9.00 16.00 1.00 29275.00  
1         19.00 12.00 1.00 17686.00  
2          8.00  7.00  NaN  7613.00  
3          7.00  3.00 1.00  2904.00  
4          2.00   NaN  NaN  1459.00  
5          2.00  1.00  NaN   807.00  
6           NaN  1.00  NaN   563.00  
7           NaN  1.00  NaN   395.00  
8           NaN   NaN  NaN   341.00  
9           NaN   NaN 1.00   306.00  
10        47.00 41.00 4.00 61349.00

### 19\. RSD 매연 농도, 종합검사 NOx허용기준별 데이터 수량

In [189]:
grp19 = die.groupby(['매연구분인덱스', '매연구분', '무부하매연허용치3'], dropna=False).agg({'차량번호':'count'}).reset_index()
grp19 = grp19.rename(columns={'무부하매연허용치3':'NOx허용기준(ppm)', '차량번호':'측정건수(건)', '매연구분':'매연(%)'})
grp19.loc[grp19.shape[0]] = [grp19['매연구분인덱스'].max() + 1, '총합', np.nan, grp19['측정건수(건)'].sum()]
grp19 = grp19.drop('매연구분인덱스', axis=1)
grp19

,매연(%),NOx허용기준(ppm),측정건수(건)
0,0이상 0.5미만,0.00,10336
1,0이상 0.5미만,8.00,645
2,0이상 0.5미만,15.00,2184
3,0이상 0.5미만,20.00,383
4,0이상 0.5미만,25.00,39
...,...,...,...
78,4.5이상,20.00,8
79,4.5이상,25.00,2
80,4.5이상,50.00,1
81,4.5이상,NaN,327


In [190]:
grp19 = die.groupby(['매연구분인덱스', '매연구분', '무부하매연허용치3'], dropna=False).agg({'차량번호':'count'}).unstack('무부하매연허용치3').reset_index()
grp19 = grp19.rename(columns={'매연구분':'매연(%)', '차량번호':'NOx허용기준(ppm)'})
grp19 = grp19.rename_axis(columns={'무부하매연허용치3':'NOx허용기준(ppm)'})
grp19['합계'] = grp19.iloc[:, 2:].sum(axis=1)
grp19.loc[grp19.shape[0]] = [grp19['매연구분인덱스'].max() + 1, '총계'] + grp19.iloc[:, 2:].sum().to_list()
grp19 = grp19.drop('매연구분인덱스', axis=1)
grp19

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15392\2668315297.py:6: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grp19 = grp19.drop('매연구분인덱스', axis=1)


매연(%) NOx허용기준(ppm)                                     \
NOx허용기준(ppm)                       0.0     8.0    15.0   20.0  25.0  30.0   
0               0이상 0.5미만     10336.00  645.00 2184.00 383.00 39.00  7.00   
1             0.5이상 1.0미만      6487.00  345.00  877.00 156.00  8.00  2.00   
2             1.0이상 1.5미만      2606.00  188.00  481.00  96.00  4.00  3.00   
3             1.5이상 2.0미만       967.00   86.00  200.00  58.00  6.00  3.00   
4             2.0이상 2.5미만       521.00   43.00  135.00  23.00  1.00   NaN   
5             2.5이상 3.0미만       272.00   34.00   73.00   9.00  3.00   NaN   
6             3.0이상 3.5미만       194.00   19.00   63.00  16.00  1.00   NaN   
7             3.5이상 4.0미만       124.00   16.00   37.00  10.00  2.00   NaN   
8             4.0이상 4.5미만       102.00    9.00   36.00  10.00  5.00   NaN   
9                   4.5이상       100.00    8.00   44.00   8.00  2.00   NaN   
10                     총계     21709.00 1393.00 4130.00 769.00 71.00 15.00   

                                                                           \
NOx허용기준(ppm) 50.0 580.0 700.0 1200.0 1240.0 1830.0 2070.0 2340.0      NaN   
0            1.00   NaN  3.00   3.00    NaN   1.00   2.00    NaN 38991.00   
1            1.00  1.00  4.00    NaN    NaN   2.00    NaN   1.00 22514.00   
2             NaN   NaN  1.00    NaN   1.00   1.00    NaN   1.00  9300.00   
3            1.00   NaN   NaN    NaN    NaN    NaN    NaN   1.00  3510.00   
4             NaN   NaN   NaN    NaN    NaN    NaN    NaN   1.00  1646.00   
5             NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   951.00   
6             NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   624.00   
7             NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   490.00   
8             NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   378.00   
9            1.00   NaN   NaN    NaN    NaN    NaN    NaN    NaN   327.00   
10           4.00  1.00  8.00   3.00   1.00   4.00   2.00   4.00 78731.00   

                    합계  
NOx허용기준(ppm)            
0             52595.00  
1             30398.00  
2             12682.00  
3              4832.00  
4              2370.00  
5              1342.00  
6               917.00  
7               679.00  
8               540.00  
9               490.00  
10           106845.00

## 종합검사 매연검사 결과와 매연RSD 매연측정 값 분석
- 매연RSD측정 매연농도 상위10%의 종합검사 매연검사 결과(적/부, 농도) 비교

In [191]:
df.shape[0]

252257

In [192]:
die.shape[0]

106845

In [193]:
top10_num = round(die.shape[0] * 0.1)
top10_num

10684

In [194]:
top10 = die.sort_values('매연', ascending=False).iloc[:top10_num]
top10.shape

(10684, 55)

### 20\. 종합검사의 매연측정결과 적합비율

In [195]:
grp20 = top10.groupby(['무부하매연판정1'], dropna=False).agg({'차량번호':'count'}).reset_index().sort_values('무부하매연판정1', ascending=False)
grp20['합계'] = grp20['차량번호'].sum()
grp20['비율'] = grp20['차량번호'] / grp20['합계']
grp20['비율'] = grp20['비율'].round(2)
grp20 = grp20.drop('합계', axis=1)
grp20 = grp20.rename(columns={'무부하매연판정1':'매연판정', '차량번호':'검사건수(건)'})
grp20.loc[grp20.shape[0]] = ['총합'] + grp20.iloc[:, 1:].sum().to_list()
grp20

,매연판정,검사건수(건),비율
1,Y,6410.00,0.60
0,N,81.00,0.01
2,NaN,4193.00,0.39
3,총합,10684.00,1.00


### [❌] 21\. 종합검사의 매연측정결과 부적합비율
- 20, 21번 한번에 처리

### [❌] (차후) 22\. 종합검사의 매연측정값 vs 매연RSD 매연값
- Q. 1대1 비교?

### 종합검사 매연 부적합 경유차 vs 매연RSD 매연값 비교

### [❌] (차후) 23\. 종합검사 매연농도 vs 매연RSD 매연농도 1:1비교

## 24\. 시험일 후 최초검사 부적합 - 매연RSD 값
- 최초 정밀검사 부적합 차량의 매연RSD 값 확인하여 오류-적합, 오류-부적합 확인
- 과정
    - 1. 유효데이터(매연제한 없음) 중 경유차 추출
    - 2. 운행차 정밀검사 기준으로 매연RSD 판정 부적합 데이터 추출
    - 3. 검사일자(정밀검사) - 검사날짜(원격측정) = 검사날짜차이
    - 4. '검사날짜차이' > 0 인 샘플 추출
    - 5. '검사날짜차이' 오름차순, 자동차등록번호 중복 제거
    - 6. 정밀검사 '무부하매연판정1' 비율

In [196]:
rcse.shape

(398673, 47)

In [197]:
rcse.columns

Index(['측정날짜', '측정시간', '기록상태', '유효', '속도', '가속도', '플래그', 'VSP', '매연', '차량번호',
       '차선번호', '풍속', '습도', '온도', '측정장소코드', '측정날짜시간', '배출가스인증번호', '배출가스등급',
       '최초등록일자', '제원관리번호', '수정일시', '차량말소YN', '차량제작일자', '차량연식', '차종', '차대번호',
       '자동차형식', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '차종분류',
       '차종유형', '무부하매연측정치1', '무부하매연판정1', '무부하매연허용치1', '무부하매연측정치2', '무부하매연판정2',
       '무부하매연허용치2', '무부하매연측정치3', '무부하매연판정3', '무부하매연허용치3', '검사방법', '검사일자'],
      dtype='object')

In [198]:
# 1. 경유차(매연제한 없음) 중 유효데이터 추출
# vdfRecordStatus = N, vdfValid = V, samFlag = V, 차량번호 : nan, 오류 값(********) 제외, 경유
sdie = rcse[(rcse['기록상태'] == 'N') & (rcse['유효'] == 'V') & (rcse['플래그'] == 'V') & (rcse['차량번호'] != '********') & (rcse['차량번호'].isnull() == False) & (rcse['연료'] == '경유')].reset_index(drop=True)
sdie.shape

(109275, 47)

In [199]:
sdie['연료'].unique()

array(['경유'], dtype=object)

In [200]:
# 2. 운행차 정밀검사 기준으로 매연RSD 판정 부적합 데이터 추출
sdf24 = sdie[sdie['매연'] > sdie['무부하매연허용치1']].reset_index(drop=True)
sdf24.shape

(388, 47)

### [출력] RSD 부적합 데이터

In [201]:
# sdf24.to_excel(os.path.join(an_fold, 'RSD부적합_경유.xlsx'), index=False)

In [203]:
sdf24[['검사일자', '측정날짜']].dtypes

검사일자    datetime64[ns]
측정날짜    datetime64[ns]
dtype: object

In [204]:
sdf24['검사일자'] = pd.to_datetime(sdf24['검사일자'])
sdf24['측정날짜'] = pd.to_datetime(sdf24['측정날짜'])
sdf24[['검사일자', '측정날짜']].dtypes

검사일자    datetime64[ns]
측정날짜    datetime64[ns]
dtype: object

In [205]:
# 3. 검사일자(정밀검사) - 측정날짜(원격측정) = 검사날짜차이
sdf24['검사날짜차이'] = sdf24['검사일자'] - sdf24['측정날짜']
sdf24['검사날짜차이'] = sdf24['검사날짜차이'].astype('str')
sdf24['검사날짜차이'] = sdf24['검사날짜차이'].str.split(' ').str[0]
sdf24['검사날짜차이'] = pd.to_numeric(sdf24['검사날짜차이'], errors='coerce')

In [206]:
# 4. '검사날짜차이' > 0 인 샘플 추출
sdf24 = sdf24[sdf24['검사날짜차이'] > 0].reset_index(drop=True)
sdf24.shape

(130, 48)

In [207]:
# 5. '검사날짜차이' 오름차순
sdf24a = sdf24.sort_values('검사날짜차이').drop_duplicates('차량번호').reset_index(drop=True)
sdf24a.shape

(126, 48)

In [209]:
sdf24a[['검사일자', '측정날짜', '검사날짜차이']].head()

,검사일자,측정날짜,검사날짜차이
0,2023-04-06,2023-04-04,2
1,2023-01-05,2023-01-03,2
2,2023-07-29,2023-07-26,3
3,2023-02-11,2023-02-06,5
4,2023-07-31,2023-07-26,5


In [210]:
# 6. 정밀검사 '무부하매연판정1' 비율
grp24 = sdf24a['무부하매연판정1'].value_counts(dropna=False, normalize=True).reset_index()
grp24 = grp24.rename(columns={'index':'매연판정', '무부하매연판정1':'판정비율'})
grp24['판정비율'] = grp24['판정비율'].round(2)
grp24.loc[grp24.shape[0]] = ['총합', round(grp24['판정비율'].sum(), 2)]
grp24

,매연판정,판정비율
0,Y,0.98
1,N,0.02
2,총합,1.00


## 25\. 시험일 후 최종검사 부적합 - 매연RSD값
- 과정
    - 1. 유효데이터(매연제한 없음) 중 경유차 추출
    - 2. 운행차 정밀검사 기준으로 매연RSD 판정 부적합 데이터 추출
    - 3. 검사일자(정밀검사) - 검사날짜(원격측정) = 검사날짜차이
    - 4. '검사날짜차이' > 0 인 샘플 추출
    - 5. '검사날짜차이' 내림차순, 자동차등록번호 중복 제거
    - 6. 정밀검사 '무부하매연판정1' 비율

In [211]:
# 1 ~ 4. 동일 과정
# 5. '검사날짜차이' 내림차순
sdf25 = sdf24.sort_values('검사날짜차이', ascending=False).drop_duplicates('차량번호').reset_index(drop=True)
sdf25.shape

(126, 48)

In [212]:
sdf25[['검사일자', '측정날짜', '검사날짜차이']].head()

,검사일자,측정날짜,검사날짜차이
0,2023-07-06,2022-10-14,265
1,2023-07-20,2022-11-01,261
2,2023-07-31,2022-11-17,256
3,2023-07-25,2022-11-17,250
4,2023-07-17,2022-11-10,249


In [213]:
# 6. 정밀검사 '무부하매연판정1' 비율
grp25 = sdf25['무부하매연판정1'].value_counts(dropna=False, normalize=True).reset_index()
grp25 = grp25.rename(columns={'index':'매연판정', '무부하매연판정1':'판정비율'})
grp25['판정비율'] = grp25['판정비율'].round(2)
grp25.loc[grp25.shape[0]] = ['총합', round(grp25['판정비율'].sum(), 2)]
grp25

,매연판정,판정비율
0,Y,0.98
1,N,0.02
2,총합,1.00


## 종합검사 NOx 부적합 경유차 vs 매연RSD 매연값 비교

### [❌] (차후) 26\. 종합검사 NOx vs 매연RSD 매연농도 1:1비교

## 고정식 다차선 매연 원격측정기 유효 측정수량 검토

### [❌] (보류) 27\. 일자별 유효 측정수량
- 1. 전체 통과 차량 건수 : UFM 사진파일 수 기록
- 2. 다차선 매연RSD측정 건수 : 해당일 매연RSD측정 유효데이터 건수
- 3. 유효율 : b번 / a번 *100(%)

## 결과 출력

In [400]:
today_date = dt.datetime.today().strftime("%Y%m%d")
today_date

'20231004'

In [401]:
with pd.ExcelWriter(os.path.join(an_fold, f'매연RSD측정결과_분석({today_date}).xlsx')) as writer:
    grp1.to_excel(writer, sheet_name='1_연료별', index=False)
    grp2.to_excel(writer, sheet_name='2_경유_차종별', index=False)
    grp3.to_excel(writer, sheet_name='3_경유_연식별', index=False)
    grp4.to_excel(writer, sheet_name='4_경유_차선별', index=False)
    grp5.to_excel(writer, sheet_name='5_경유_속도별', index=False)
    grp6.to_excel(writer, sheet_name='6_경유_배출가스등급별', index=False)
    grp7.to_excel(writer, sheet_name='7_경유_배기량별', index=False)
    grp8.to_excel(writer, sheet_name='8_경유_차량중량별', index=False)
    grp9.to_excel(writer, sheet_name='9_경유_매연허용기준별', index=False)
    grp10.to_excel(writer, sheet_name='10_경유_NOx허용기준별', index=False)

    grp11.to_excel(writer, sheet_name='11_농도별', index=False)
    grp12.to_excel(writer, sheet_name='12_농도_차종별', index=True)
    grp13.to_excel(writer, sheet_name='13_농도_연식별', index=True)
    grp14.to_excel(writer, sheet_name='14_농도_측정속도별', index=True)
    grp15.to_excel(writer, sheet_name='15_농도_배출가스등급별', index=True)
    grp16.to_excel(writer, sheet_name='16_농도_배기량별', index=True)
    grp17.to_excel(writer, sheet_name='17_농도_차량중량별', index=True)
    grp18.to_excel(writer, sheet_name='18_농도_매연허용기준별', index=True)
    grp19.to_excel(writer, sheet_name='19_농도_NOx허용기준별', index=True)

    grp20.to_excel(writer, sheet_name='20_매연_적합_부적합비율', index=False)
    # grp21.to_excel(writer, sheet_name='20_매연_적합_부적합비율', index=False) # 20, 21번 한번에 처리
    # grp22.to_excel(writer, sheet_name='22_매연측정값과_매연RSD비교', index=False) # 어떤식으로 비교하는지 문의 필요

    # grp23.to_excel(writer, sheet_name='23_매연농도와_매연RSD_1대1비교', index=False) # 차후 논의
    
    grp24.to_excel(writer, sheet_name='24_시험일후_최초검사_부적합', index=False) # 문의 필요
    grp25.to_excel(writer, sheet_name='25_시험일후_최종검사_부적합', index=False) # 문의 필요

    # grp26.to_excel(writer, sheet_name='26_종합검사NOx와_RSD매연_1대1비교', index=False) # 차후 논의

    # grp27.to_excel(writer, sheet_name='27_일자별_유효_측정수량', index=False) # 보류

In [402]:
sec = time.time() - start_time
print(str(dt.timedelta(seconds=sec)).split('.')[0])

3:33:47


# End